## Interactive Brokers Day Trading

An Interactive Brokers (IB) algorithm bot for forex day trading.

### Import Dependencies and Setup

In [ ]:
# Import required libraries
import os
import asyncio
import logging
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from typing import Dict, List, Optional
import time

# Interactive Brokers imports
from ib_insync import IB, Crypto, MarketOrder, LimitOrder, Ticker
from ib_insync import util

log_dir = "logs"
log_file = "forex_trading_bot.log"
os.makedirs(log_dir, exist_ok=True)                  # create folder if missing
log_path = os.path.join(log_dir, log_file)

# touch the file if it doesn't exist
if not os.path.exists(log_path):
    open(log_path, "a").close()

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(),
        logging.FileHandler(log_path)
    ]
)

# Suppress ib_insync debug logs
logging.getLogger('ib_insync').setLevel(logging.WARNING)

print("✓ Dependencies imported successfully")
print("✓ Logging configured")

### Environment Configuration

In [ ]:
class CryptoEnvironment:
    """Configuration for IB crypto trading environments"""
    
    # Paper Trading Configuration
    PAPER_TRADING = {
        'host': '127.0.0.1',
        'port': 7497,
        'client_id': 1,
        'description': 'Paper Trading (Demo Account)'
    }
    
    # Live Trading Configuration
    LIVE_TRADING = {
        'host': '127.0.0.1',
        'port': 7496,
        'client_id': 2,
        'description': 'Live Trading (Real Money)'
    }
    
    @classmethod
    def get_config(cls, environment: str = 'paper') -> Dict:
        """Get configuration for specified environment"""
        if environment.lower() == 'paper':
            return cls.PAPER_TRADING.copy()
        elif environment.lower() == 'live':
            return cls.LIVE_TRADING.copy()
        else:
            raise ValueError("Environment must be 'paper' or 'live'")
    
    @classmethod
    def list_environments(cls):
        """List available trading environments"""
        print("Available Trading Environments:")
        print(f"  paper: {cls.PAPER_TRADING['description']} - Port {cls.PAPER_TRADING['port']}")
        print(f"  live:  {cls.LIVE_TRADING['description']} - Port {cls.LIVE_TRADING['port']}")

CryptoEnvironment.list_environments()
print("\n✓ Environment configuration ready")

### IB Connection Handler

In [ ]:
class CryptoIBConnection:
    """Interactive Brokers connection handler for cryptocurrency trading"""
    
    def __init__(self, environment: str = 'paper'):
        """Initialise IB connection for crypto trading"""
        self.config = CryptoEnvironment.get_config(environment)
        self.environment = environment
        self.ib = IB()
        self.connected = False
        self.logger = logging.getLogger(f'CryptoIB-{environment}')
        
        # Connection parameters with unique client IDs
        self.host = self.config['host']
        self.port = self.config['port']
        # Use timestamp to ensure unique client ID
        self.client_id = self.config['client_id'] + int(time.time() % 1000)
        
        self.logger.info(f"Initialised for {self.config['description']}")
    
    def connect(self, timeout: int = 10) -> bool:
        """Connect to Interactive Brokers TWS or Gateway"""
        try:
            self.logger.info(f"Connecting to {self.config['description']}...")
            
            # Handle Jupyter notebook event loop
            try:
                import IPython
                if IPython.get_ipython() is not None:
                    # We're in Jupyter - use util.startLoop for compatibility
                    util.startLoop()
            except ImportError:
                pass  # Not in Jupyter
            
            # Connect with unique client ID
            self.ib.connect(
                host=self.host,
                port=self.port,
                clientId=self.client_id,
                timeout=timeout
            )
            
            self.connected = True
            self.logger.info(f"✓ Connected to IB on port {self.port} (Client ID: {self.client_id})")
            
            # Get account information
            try:
                account = self.ib.managedAccounts()[0] if self.ib.managedAccounts() else "Unknown"
                self.logger.info(f"Account: {account}")
            except:
                pass
            
            return True
            
        except Exception as e:
            if "already running" in str(e):
                # Try alternative connection method for Jupyter
                return self._connect_jupyter_compatible(timeout)
            else:
                self.logger.error(f"Connection failed: {e}")
                if "client id is already in use" in str(e):
                    # Try with different client ID
                    self.client_id += 10
                    self.logger.info(f"Retrying with client ID: {self.client_id}")
                    return self._retry_connection(timeout)
                
                self.logger.error("Ensure TWS or IB Gateway is running with API enabled")
                self.connected = False
                return False
    
    def _connect_jupyter_compatible(self, timeout: int = 10) -> bool:
        """Alternative connection method for Jupyter notebooks"""
        try:
            # Create fresh IB instance to avoid event loop conflicts
            self.ib.disconnect()  # Ensure clean state
            time.sleep(1)
            
            self.ib = IB()
            
            # Try connecting without event loop management
            self.ib.connect(
                host=self.host,
                port=self.port,
                clientId=self.client_id,
                timeout=timeout
            )
            
            self.connected = True
            self.logger.info(f"✓ Connected via Jupyter method (Client ID: {self.client_id})")
            return True
            
        except Exception as e:
            if "client id is already in use" in str(e):
                self.client_id += 20
                return self._retry_connection(timeout)
            
            self.logger.error(f"Jupyter connection failed: {e}")
            self.connected = False
            return False
    
    def _retry_connection(self, timeout: int = 10) -> bool:
        """Retry connection with new client ID"""
        try:
            self.ib.disconnect()
            time.sleep(2)
            
            self.ib = IB()
            
            self.ib.connect(
                host=self.host,
                port=self.port,
                clientId=self.client_id,
                timeout=timeout
            )
            
            self.connected = True
            self.logger.info(f"✓ Connected with retry (Client ID: {self.client_id})")
            return True
            
        except Exception as e:
            self.logger.error(f"Retry connection failed: {e}")
            self.connected = False
            return False
    
    def disconnect(self):
        """Disconnect from Interactive Brokers"""
        try:
            if self.connected and self.ib.isConnected():
                self.ib.disconnect()
                self.logger.info("Disconnected from IB")
                time.sleep(1)  # Allow clean disconnect
        except Exception as e:
            self.logger.error(f"Error during disconnect: {e}")
        finally:
            self.connected = False
    
    def is_connected(self) -> bool:
        """Check if connection is active"""
        try:
            return self.connected and self.ib.isConnected()
        except:
            return False

print("✓ Crypto IB Connection class created (Jupyter compatible)")

### Market Data Handler

In [ ]:
class ForexMarketData:
    """Handle forex market data from Interactive Brokers (Crypto alternative)"""
    
    def __init__(self, ib_connection: CryptoIBConnection):
        """Initialise forex market data handler"""
        self.ib_conn = ib_connection
        self.ib = ib_connection.ib
        self.logger = logging.getLogger('ForexMarketData')
        self.last_prices = {}

        # Major and minor forex pairs available on Interactive Brokers
        self.forex_pairs = {
            # Major pairs
            'EURUSD': 'EUR/USD',
            'GBPUSD': 'GBP/USD',
            'USDJPY': 'USD/JPY',
            'NZDUSD': 'NZD/USD',
            'AUDUSD': 'AUD/USD',
            'USDCAD': 'USD/CAD',
            'USDCHF': 'USD/CHF',
            
            # Minor pairs (cross currencies)
            'EURGBP': 'EUR/GBP',
            'EURJPY': 'EUR/JPY',
            'EURCHF': 'EUR/CHF',
            'EURCAD': 'EUR/CAD',
            'EURAUD': 'EUR/AUD',
            'EURNZD': 'EUR/NZD',
            'GBPJPY': 'GBP/JPY',
            'GBPCHF': 'GBP/CHF',
            'GBPCAD': 'GBP/CAD',
            'GBPAUD': 'GBP/AUD',
            'GBPNZD': 'GBP/NZD',
            'AUDJPY': 'AUD/JPY',
            'AUDCHF': 'AUD/CHF',
            'AUDCAD': 'AUD/CAD',
            'AUDNZD': 'AUD/NZD',
            'CHFJPY': 'CHF/JPY',
            'CADJPY': 'CAD/JPY',
            'NZDJPY': 'NZD/JPY',
            'NZDCHF': 'NZD/CHF',
            'NZDCAD': 'NZD/CAD',
            'CADCHF': 'CAD/CHF',
            
            # Emerging market pairs
            'USDMXN': 'USD/MXN',
            'USDZAR': 'USD/ZAR',
            'USDSGD': 'USD/SGD',
            'USDHKD': 'USD/HKD',
            'USDNOK': 'USD/NOK',
            'USDSEK': 'USD/SEK',
            'USDDKK': 'USD/DKK',
            'USDPLN': 'USD/PLN',
            'USDCZK': 'USD/CZK',
            'USDHUF': 'USD/HUF',
            'USDTRY': 'USD/TRY',
            'USDRUB': 'USD/RUB',
            'USDBRL': 'USD/BRL',
            'USDCNH': 'USD/CNH',
            'USDINR': 'USD/INR',
            'USDKRW': 'USD/KRW',
            'EURPLN': 'EUR/PLN',
            'EURNOK': 'EUR/NOK',
            'EURSEK': 'EUR/SEK',
            'EURDKK': 'EUR/DKK',
            'EURCZK': 'EUR/CZK',
            'EURHUF': 'EUR/HUF',
            'EURTRY': 'EUR/TRY',
        }
    
    def create_forex_contract(self, pair: str):
        """Create forex contract for IB"""
        from ib_insync import Forex
        return Forex(pair)
    
    def test_forex_contract(self, pair: str) -> bool:
        """Test if forex contract is valid"""
        if not self.ib_conn.is_connected():
            return False
        
        try:
            contract = self.create_forex_contract(pair)
            details = self.ib.reqContractDetails(contract)
            
            if details:
                self.logger.info(f"✓ {pair} contract validated")
                return True
            else:
                self.logger.warning(f"No contract details for {pair}")
                return False
                
        except Exception as e:
            self.logger.error(f"Contract test failed for {pair}: {e}")
            return False
    
    def get_forex_price(self, pair: str) -> Optional[Dict]:
        """Get current forex price"""
        if not self.ib_conn.is_connected():
            self.logger.error("Not connected to IB")
            return None
        
        try:
            contract = self.create_forex_contract(pair)
            
            # Request market data
            ticker = self.ib.reqMktData(contract, '', False, False)
            self.ib.sleep(2)
            
            if ticker and ticker.last and ticker.last > 0:
                forex_data = {
                    'symbol': pair,
                    'last': float(ticker.last),
                    'bid': float(ticker.bid) if ticker.bid and ticker.bid > 0 else None,
                    'ask': float(ticker.ask) if ticker.ask and ticker.ask > 0 else None,
                    'timestamp': datetime.now()
                }
                
                self.last_prices[pair] = forex_data['last']
                self.logger.info(f"{pair} rate: {forex_data['last']:.5f}")
                return forex_data
            else:
                self.logger.warning(f"No market data for {pair}")
                return None
                
        except Exception as e:
            self.logger.error(f"Error getting {pair} price: {e}")
            return None
        finally:
            try:
                self.ib.cancelMktData(contract)
            except:
                pass
    
    def get_available_pairs(self) -> List[str]:
        """Get list of available forex pairs by testing contracts"""
        available = []
        
        for pair in self.forex_pairs.keys():
            if self.test_forex_contract(pair):
                available.append(pair)
                time.sleep(0.5)
        
        return available
    
    def get_historical_forex_data(self, pair: str, duration: str = '1 D',
                                 bar_size: str = '5 mins') -> Optional[pd.DataFrame]:
        """Get historical forex data"""
        if not self.ib_conn.is_connected():
            self.logger.error("Not connected to IB")
            return None
        
        try:
            contract = self.create_forex_contract(pair)
            
            bars = self.ib.reqHistoricalData(
                contract,
                endDateTime='',
                durationStr=duration,
                barSizeSetting=bar_size,
                whatToShow='MIDPOINT',
                useRTH=False,  # 24/7 forex trading
                formatDate=1
            )
            
            if bars:
                df = util.df(bars)
                df['timestamp'] = pd.to_datetime(df['date'])
                df['symbol'] = pair
                
                self.logger.info(f"Retrieved {len(df)} historical bars for {pair}")
                return df
            else:
                self.logger.warning(f"No historical data for {pair}")
                return None
                
        except Exception as e:
            self.logger.error(f"Error getting historical data for {pair}: {e}")
            return None
    
    def get_supported_pairs(self) -> List[str]:
        """Get list of supported forex pairs"""
        return list(self.forex_pairs.keys())

print("✓ Forex Market Data Handler created (Working alternative)")

### Order Management

In [ ]:
class ForexOrderManager:
    """Handle forex order placement and management"""
    
    def __init__(self, ib_connection: CryptoIBConnection):
        """Initialise forex order manager"""
        self.ib_conn = ib_connection
        self.ib = ib_connection.ib
        self.logger = logging.getLogger('ForexOrderManager')
        self.orders_log = []
    
    def place_forex_market_order(self, pair: str, action: str, quantity: int) -> bool:
        """Place a forex market order"""
        if not self.ib_conn.is_connected():
            self.logger.error("Not connected to IB")
            return False
        
        try:
            from ib_insync import Forex
            
            contract = Forex(pair)
            order = MarketOrder(action, quantity)
            
            # Place the order
            trade = self.ib.placeOrder(contract, order)
            
            if trade:
                order_info = {
                    'symbol': pair,
                    'action': action,
                    'quantity': quantity,
                    'order_type': 'MARKET',
                    'status': trade.orderStatus.status,
                    'timestamp': datetime.now(),
                    'order_id': trade.order.orderId
                }
                
                self.orders_log.append(order_info)
                self.logger.info(f"✓ Forex market order: {action} {quantity} {pair}")
                return True
            else:
                self.logger.error(f"Failed to place market order for {pair}")
                return False
                
        except Exception as e:
            self.logger.error(f"Error placing forex order for {pair}: {e}")
            return False
    
    def get_orders_log(self) -> List[Dict]:
        """Get log of all placed orders"""
        return self.orders_log

print("✓ Forex Order Manager created")

### Trading Strategy

In [ ]:
class AdvancedForexDayTradingStrategy:
    """Advanced forex day trading strategy optimised for maximum profit"""
    
    def __init__(self, name: str = "AdvancedForexDay"):
        """Initialise advanced forex strategy"""
        self.name = name
        self.logger = logging.getLogger(f'AdvancedForexStrategy-{name}')
        
        # Advanced strategy parameters
        self.rsi_oversold = 25
        self.rsi_overbought = 75
        self.momentum_threshold = 0.08  # More sensitive
        self.volatility_threshold = 0.5  # Pip volatility threshold
        
        # Multi-timeframe analysis
        self.short_ma = 8
        self.medium_ma = 21
        self.long_ma = 50
        
        # Position tracking with advanced metrics
        self.positions = {}
        self.trade_history = []
        
        # Dynamic risk management
        self.base_position_pct = 0.08  # 8% base position
        self.max_position_pct = 0.25   # 25% maximum exposure
        self.dynamic_stop_loss = True
        self.base_stop_pips = 15
        self.base_profit_pips = 30
        
        # Performance tracking
        self.trades_count = 0
        self.successful_trades = 0
        self.total_pnl = 0.0
        self.max_drawdown = 0.0
        self.peak_equity = 0.0
        
        # Market regime detection
        self.market_regime = 'trending'  # 'trending', 'ranging', 'volatile'
        self.regime_history = []
        
        # News and session awareness
        self.session_weights = {
            'asian': 0.7,     # Lower volatility
            'london': 1.2,    # High volatility
            'newyork': 1.0,   # Standard
            'overlap': 1.5    # Highest volatility
        }
    
    def get_trading_session(self) -> str:
        """Determine current trading session"""
        utc_hour = datetime.utcnow().hour
        
        # Trading session times (UTC)
        if 0 <= utc_hour < 7:
            return 'asian'
        elif 7 <= utc_hour < 12:
            return 'london'
        elif 12 <= utc_hour < 17:
            return 'overlap'  # London-NY overlap
        elif 17 <= utc_hour < 22:
            return 'newyork'
        else:
            return 'asian'
    
    def calculate_advanced_indicators(self, df: pd.DataFrame) -> Dict:
        """Calculate technical indicators for forex"""
        indicators = {}
        
        if df is None or len(df) < 50:
            return indicators
        
        try:
            # Multi-timeframe moving averages
            indicators['sma_8'] = df['close'].rolling(window=8).mean().iloc[-1]
            indicators['sma_21'] = df['close'].rolling(window=21).mean().iloc[-1]
            indicators['sma_50'] = df['close'].rolling(window=50).mean().iloc[-1]
            
            # Exponential moving averages
            indicators['ema_8'] = df['close'].ewm(span=8).mean().iloc[-1]
            indicators['ema_21'] = df['close'].ewm(span=21).mean().iloc[-1]
            
            # Advanced RSI with multiple periods
            for period in [14, 21]:
                delta = df['close'].diff()
                gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
                loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
                rs = gain / loss
                indicators[f'rsi_{period}'] = 100 - (100 / (1 + rs.iloc[-1]))
            
            # MACD with signal line
            ema_12 = df['close'].ewm(span=12).mean()
            ema_26 = df['close'].ewm(span=26).mean()
            macd_line = ema_12 - ema_26
            indicators['macd'] = macd_line.iloc[-1]
            indicators['macd_signal'] = macd_line.ewm(span=9).mean().iloc[-1]
            indicators['macd_histogram'] = indicators['macd'] - indicators['macd_signal']
            
            # Bollinger Bands with multiple periods
            for period in [20, 50]:
                sma = df['close'].rolling(window=period).mean()
                std = df['close'].rolling(window=period).std()
                indicators[f'bb_{period}_upper'] = (sma + (std * 2)).iloc[-1]
                indicators[f'bb_{period}_lower'] = (sma - (std * 2)).iloc[-1]
                indicators[f'bb_{period}_middle'] = sma.iloc[-1]
            
            # Stochastic Oscillator
            if len(df) >= 14:
                low_14 = df['low'].rolling(window=14).min()
                high_14 = df['high'].rolling(window=14).max()
                k_percent = 100 * ((df['close'] - low_14) / (high_14 - low_14))
                indicators['stoch_k'] = k_percent.iloc[-1]
                indicators['stoch_d'] = k_percent.rolling(window=3).mean().iloc[-1]
            
            # Average True Range (ATR) for volatility
            high_low = df['high'] - df['low']
            high_close = abs(df['high'] - df['close'].shift())
            low_close = abs(df['low'] - df['close'].shift())
            true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
            indicators['atr'] = true_range.rolling(window=14).mean().iloc[-1]
            
            # Pip-based calculations
            pair_symbol = df['symbol'].iloc[0] if 'symbol' in df.columns else 'EURUSD'
            pip_size = 0.0001 if 'JPY' not in pair_symbol else 0.01
            
            # Recent pip movements
            indicators['pip_movement_1'] = (df['close'].iloc[-1] - df['close'].iloc[-2]) / pip_size
            indicators['pip_movement_5'] = (df['close'].iloc[-1] - df['close'].iloc[-6]) / pip_size
            indicators['pip_range'] = (df['high'].iloc[-1] - df['low'].iloc[-1]) / pip_size
            
            # Momentum indicators
            if len(df) >= 10:
                indicators['momentum_3'] = ((df['close'].iloc[-1] - df['close'].iloc[-4]) / df['close'].iloc[-4]) * 100
                indicators['momentum_10'] = ((df['close'].iloc[-1] - df['close'].iloc[-11]) / df['close'].iloc[-11]) * 100
            
            # Trend strength
            if len(df) >= 21:
                price = df['close'].iloc[-1]
                sma_21 = indicators['sma_21']
                indicators['trend_strength'] = abs((price - sma_21) / sma_21) * 100
                indicators['trend_direction'] = 'bullish' if price > sma_21 else 'bearish'
            
            # Volume-price analysis (if volume available)
            if 'volume' in df.columns and df['volume'].sum() > 0:
                indicators['volume_trend'] = df['volume'].rolling(window=10).mean().iloc[-1]
                indicators['volume_current'] = df['volume'].iloc[-1]
            
            # Support and resistance levels
            if len(df) >= 50:
                recent_period = 20
                indicators['resistance'] = df['high'].rolling(window=recent_period).max().iloc[-1]
                indicators['support'] = df['low'].rolling(window=recent_period).min().iloc[-1]
                
                # Distance to S/R levels
                current_price = df['close'].iloc[-1]
                indicators['distance_to_resistance'] = ((indicators['resistance'] - current_price) / pip_size)
                indicators['distance_to_support'] = ((current_price - indicators['support']) / pip_size)
            
        except Exception as e:
            self.logger.error(f"Error calculating advanced indicators: {e}")
        
        return indicators
    
    def detect_market_regime(self, df: pd.DataFrame, indicators: Dict) -> str:
        """Detect current market regime for strategy adaptation"""
        try:
            if len(df) < 50:
                return 'trending'
            
            # Calculate regime indicators
            atr = indicators.get('atr', 0)
            trend_strength = indicators.get('trend_strength', 0)
            
            # Price range analysis
            recent_high = df['high'].rolling(window=20).max().iloc[-1]
            recent_low = df['low'].rolling(window=20).min().iloc[-1]
            current_price = df['close'].iloc[-1]
            
            range_position = (current_price - recent_low) / (recent_high - recent_low)
            
            # Regime classification
            if trend_strength > 1.0 and atr > 0.001:
                regime = 'trending'
            elif range_position > 0.3 and range_position < 0.7 and atr < 0.0008:
                regime = 'ranging'
            elif atr > 0.0015:
                regime = 'volatile'
            else:
                regime = 'trending'
            
            # Store regime history
            self.regime_history.append({
                'timestamp': datetime.now(),
                'regime': regime,
                'atr': atr,
                'trend_strength': trend_strength
            })
            
            # Keep last 100 regime readings
            if len(self.regime_history) > 100:
                self.regime_history = self.regime_history[-100:]
            
            self.market_regime = regime
            return regime
            
        except Exception as e:
            self.logger.error(f"Error detecting market regime: {e}")
            return 'trending'
    
    def calculate_dynamic_position_size(self, pair: str, indicators: Dict, 
                                      available_capital: float) -> int:
        """Calculate dynamic position size based on market conditions"""
        base_size = available_capital * self.base_position_pct
        
        # Session-based adjustment
        session = self.get_trading_session()
        session_multiplier = self.session_weights.get(session, 1.0)
        
        # Volatility adjustment
        atr = indicators.get('atr', 0.001)
        if atr > 0.002:  # High volatility
            volatility_multiplier = 0.7
        elif atr < 0.0005:  # Low volatility
            volatility_multiplier = 1.3
        else:
            volatility_multiplier = 1.0
        
        # Trend strength adjustment
        trend_strength = indicators.get('trend_strength', 0)
        if trend_strength > 2.0:  # Strong trend
            trend_multiplier = 1.4
        elif trend_strength < 0.5:  # Weak trend
            trend_multiplier = 0.8
        else:
            trend_multiplier = 1.0
        
        # Market regime adjustment
        regime_multiplier = {
            'trending': 1.2,
            'ranging': 0.8,
            'volatile': 0.6
        }.get(self.market_regime, 1.0)
        
        # Calculate final position size
        adjusted_size = (base_size * session_multiplier * volatility_multiplier * 
                        trend_multiplier * regime_multiplier)
        
        # Apply maximum limit
        max_size = available_capital * self.max_position_pct
        final_size = min(adjusted_size, max_size)
        
        # Convert to standard lot sizes
        if final_size >= 1000:
            return 1000  # Standard lot
        elif final_size >= 100:
            return 100   # Mini lot
        elif final_size >= 1000:
            return 1000    # Micro lot
        else:
            return 100     # Nano lot
    
    def calculate_dynamic_stops(self, pair: str, indicators: Dict, 
                               entry_price: float) -> Dict:
        """Calculate dynamic stop loss and take profit levels"""
        atr = indicators.get('atr', 0.001)
        pip_size = 0.0001 if 'JPY' not in pair else 0.01
        
        # ATR-based stops
        atr_pips = atr / pip_size
        
        # Dynamic stop loss (1.5x to 3x ATR)
        if self.market_regime == 'volatile':
            stop_multiplier = 3.0
        elif self.market_regime == 'trending':
            stop_multiplier = 2.0
        else:  # ranging
            stop_multiplier = 1.5
        
        dynamic_stop_pips = max(self.base_stop_pips, atr_pips * stop_multiplier)
        
        # Dynamic take profit (2:1 to 4:1 risk-reward)
        if self.market_regime == 'trending':
            profit_ratio = 4.0
        elif self.market_regime == 'volatile':
            profit_ratio = 2.5
        else:  # ranging
            profit_ratio = 2.0
        
        dynamic_profit_pips = dynamic_stop_pips * profit_ratio
        
        # Support/resistance adjustment
        if 'distance_to_resistance' in indicators and 'distance_to_support' in indicators:
            resistance_distance = indicators['distance_to_resistance']
            support_distance = indicators['distance_to_support']
            
            # Adjust profit target if resistance/support is closer
            if resistance_distance < dynamic_profit_pips:
                dynamic_profit_pips = resistance_distance * 0.8
            if support_distance < dynamic_stop_pips:
                dynamic_stop_pips = support_distance * 0.8
        
        return {
            'stop_loss_pips': dynamic_stop_pips,
            'take_profit_pips': dynamic_profit_pips,
            'risk_reward_ratio': dynamic_profit_pips / dynamic_stop_pips
        }
    
    def evaluate_advanced_buy_signals(self, pair: str, market_data: Dict, indicators: Dict) -> Dict:
        """Evaluate advanced buy signals with confidence scoring"""
        price = market_data['last']
        signals = {}
        confidence_score = 0
        
        # Signal 1: Multi-timeframe MA alignment (High weight)
        ma_alignment = False
        if all(key in indicators for key in ['sma_8', 'sma_21', 'sma_50']):
            ma_alignment = (indicators['sma_8'] > indicators['sma_21'] > indicators['sma_50'])
            signals['ma_alignment'] = ma_alignment
            if ma_alignment:
                confidence_score += 25
        
        # Signal 2: RSI multi-period confluence
        rsi_signal = False
        if 'rsi_14' in indicators and 'rsi_21' in indicators:
            rsi_14 = indicators['rsi_14']
            rsi_21 = indicators['rsi_21']
            rsi_signal = (20 < rsi_14 < 45) and (25 < rsi_21 < 50)
            signals['rsi_confluence'] = rsi_signal
            if rsi_signal:
                confidence_score += 20
        
        # Signal 3: MACD momentum
        macd_signal = False
        if 'macd' in indicators and 'macd_signal' in indicators:
            macd_bullish = (indicators['macd'] > indicators['macd_signal'] and 
                           indicators['macd_histogram'] > 0)
            signals['macd_momentum'] = macd_bullish
            if macd_bullish:
                confidence_score += 15
        
        # Signal 4: Stochastic oversold recovery
        stoch_signal = False
        if 'stoch_k' in indicators and 'stoch_d' in indicators:
            stoch_signal = (indicators['stoch_k'] > 20 and indicators['stoch_k'] < 50 and
                           indicators['stoch_k'] > indicators['stoch_d'])
            signals['stochastic'] = stoch_signal
            if stoch_signal:
                confidence_score += 15
        
        # Signal 5: Bollinger Bands position
        bb_signal = False
        if 'bb_20_lower' in indicators and 'bb_20_middle' in indicators:
            bb_signal = (price > indicators['bb_20_lower'] and 
                        price < indicators['bb_20_middle'])
            signals['bollinger_position'] = bb_signal
            if bb_signal:
                confidence_score += 10
        
        # Signal 6: Support level bounce
        support_signal = False
        if 'distance_to_support' in indicators:
            support_distance = indicators['distance_to_support']
            support_signal = 5 < support_distance < 25  # Near support but not too close
            signals['support_bounce'] = support_signal
            if support_signal:
                confidence_score += 10
        
        # Signal 7: Session-based momentum
        session = self.get_trading_session()
        session_signal = False
        if 'momentum_3' in indicators:
            momentum = indicators['momentum_3']
            if session in ['london', 'overlap']:
                session_signal = momentum > 0.05  # More aggressive in high-vol sessions
            else:
                session_signal = momentum > 0.03  # Conservative in low-vol sessions
            signals['session_momentum'] = session_signal
            if session_signal:
                confidence_score += 5
        
        return {
            'signals': signals,
            'confidence_score': confidence_score,
            'signal_count': sum(signals.values()),
            'total_signals': len(signals)
        }
    
    def evaluate_advanced_sell_signals(self, pair: str, market_data: Dict, indicators: Dict) -> Dict:
        """Evaluate advanced sell signals with confidence scoring"""
        price = market_data['last']
        signals = {}
        confidence_score = 0
        
        # Signal 1: MA breakdown
        ma_breakdown = False
        if all(key in indicators for key in ['sma_8', 'sma_21']):
            ma_breakdown = indicators['sma_8'] < indicators['sma_21']
            signals['ma_breakdown'] = ma_breakdown
            if ma_breakdown:
                confidence_score += 25
        
        # Signal 2: RSI overbought
        rsi_signal = False
        if 'rsi_14' in indicators:
            rsi_signal = indicators['rsi_14'] > 75
            signals['rsi_overbought'] = rsi_signal
            if rsi_signal:
                confidence_score += 20
        
        # Signal 3: MACD bearish divergence
        macd_signal = False
        if 'macd' in indicators and 'macd_signal' in indicators:
            macd_bearish = (indicators['macd'] < indicators['macd_signal'] and
                           indicators['macd_histogram'] < 0)
            signals['macd_bearish'] = macd_bearish
            if macd_bearish:
                confidence_score += 15
        
        # Signal 4: Bollinger Band rejection
        bb_signal = False
        if 'bb_20_upper' in indicators:
            bb_signal = price > indicators['bb_20_upper'] * 0.998
            signals['bb_rejection'] = bb_signal
            if bb_signal:
                confidence_score += 15
        
        # Signal 5: Stochastic overbought
        stoch_signal = False
        if 'stoch_k' in indicators:
            stoch_signal = indicators['stoch_k'] > 80
            signals['stoch_overbought'] = stoch_signal
            if stoch_signal:
                confidence_score += 10
        
        # Signal 6: Resistance rejection
        resistance_signal = False
        if 'distance_to_resistance' in indicators:
            resistance_distance = indicators['distance_to_resistance']
            resistance_signal = 0 < resistance_distance < 15  # Very close to resistance
            signals['resistance_rejection'] = resistance_signal
            if resistance_signal:
                confidence_score += 15
        
        return {
            'signals': signals,
            'confidence_score': confidence_score,
            'signal_count': sum(signals.values()),
            'total_signals': len(signals)
        }
    
    def should_buy_forex(self, pair: str, market_data: Dict, indicators: Dict) -> bool:
        """Advanced buy decision with confidence-based logic"""
        # Check existing position
        if self.get_position(pair)['quantity'] > 0:
            return False
        
        # Detect market regime
        regime = self.detect_market_regime(pd.DataFrame(), indicators)
        
        # Get buy signals
        buy_analysis = self.evaluate_advanced_buy_signals(pair, market_data, indicators)
        
        # Confidence thresholds by regime
        confidence_thresholds = {
            'trending': 60,    # Lower threshold in trending markets
            'ranging': 75,     # Higher threshold in ranging markets
            'volatile': 80     # Highest threshold in volatile markets
        }
        
        required_confidence = confidence_thresholds.get(regime, 70)
        decision = buy_analysis['confidence_score'] >= required_confidence
        
        if decision:
            self.logger.info(f"🟢 BUY {pair}: {buy_analysis['confidence_score']}% confidence "
                           f"({buy_analysis['signal_count']}/{buy_analysis['total_signals']} signals) "
                           f"[{regime.upper()} regime]")
        
        return decision
    
    def should_sell_forex(self, pair: str, market_data: Dict, indicators: Dict) -> bool:
        """Advanced sell decision with risk management"""
        position = self.get_position(pair)
        if position['quantity'] <= 0:
            return False
        
        price = market_data['last']
        entry_price = position['avg_price']
        
        # Calculate current P&L in pips
        pip_size = 0.0001 if 'JPY' not in pair else 0.01
        current_pnl_pips = (price - entry_price) / pip_size
        
        # Dynamic stop/profit levels
        stop_profit_levels = self.calculate_dynamic_stops(pair, indicators, entry_price)
        
        # Risk management exits (highest priority)
        if current_pnl_pips <= -stop_profit_levels['stop_loss_pips']:
            self.logger.warning(f"🔴 STOP LOSS {pair}: {current_pnl_pips:.1f} pips")
            return True
        
        if current_pnl_pips >= stop_profit_levels['take_profit_pips']:
            self.logger.info(f"🟢 TAKE PROFIT {pair}: {current_pnl_pips:.1f} pips")
            return True
        
        # Technical sell signals
        sell_analysis = self.evaluate_advanced_sell_signals(pair, market_data, indicators)
        
        # Session-based exit logic
        session = self.get_trading_session()
        if session == 'asian' and current_pnl_pips > 10:  # Take smaller profits in quiet session
            return True
        
        # Confidence-based exit
        regime = self.market_regime
        confidence_thresholds = {
            'trending': 70,
            'ranging': 60,
            'volatile': 50
        }
        
        required_confidence = confidence_thresholds.get(regime, 60)
        decision = sell_analysis['confidence_score'] >= required_confidence
        
        if decision:
            self.logger.info(f"🔴 SELL {pair}: {sell_analysis['confidence_score']}% confidence "
                           f"({sell_analysis['signal_count']}/{sell_analysis['total_signals']} signals)")
        
        return decision
    
    def update_position(self, pair: str, action: str, quantity: int, price: float):
        """Update position with advanced tracking"""
        if pair not in self.positions:
            self.positions[pair] = {
                'quantity': 0, 
                'avg_price': 0.0, 
                'entry_time': None,
                'max_profit_pips': 0.0,
                'max_loss_pips': 0.0
            }
        
        position = self.positions[pair]
        pip_size = 0.0001 if 'JPY' not in pair else 0.01
        
        if action == 'BUY':
            total_cost = (position['quantity'] * position['avg_price']) + (quantity * price)
            position['quantity'] += quantity
            position['avg_price'] = total_cost / position['quantity'] if position['quantity'] > 0 else 0
            position['entry_time'] = datetime.now()
            position['max_profit_pips'] = 0.0
            position['max_loss_pips'] = 0.0
            
        elif action == 'SELL':
            if position['quantity'] > 0:
                # Calculate trade P&L
                trade_pnl = (price - position['avg_price']) * quantity
                trade_pnl_pips = (price - position['avg_price']) / pip_size
                
                # Calculate trade duration
                trade_duration = None
                if position['entry_time']:
                    trade_duration = (datetime.now() - position['entry_time']).total_seconds() / 60
                
                # Record trade
                trade_record = {
                    'pair': pair,
                    'entry_price': position['avg_price'],
                    'exit_price': price,
                    'quantity': quantity,
                    'pnl_usd': trade_pnl,
                    'pnl_pips': trade_pnl_pips,
                    'duration_minutes': trade_duration,
                    'entry_time': position['entry_time'],
                    'exit_time': datetime.now(),
                    'regime': self.market_regime,
                    'session': self.get_trading_session()
                }
                
                self.trade_history.append(trade_record)
                
                # Update performance
                self.total_pnl += trade_pnl
                self.trades_count += 1
                if trade_pnl > 0:
                    self.successful_trades += 1
                
                # Update drawdown tracking
                self.peak_equity = max(self.peak_equity, self.total_pnl)
                current_drawdown = self.peak_equity - self.total_pnl
                self.max_drawdown = max(self.max_drawdown, current_drawdown)
                
                # Reset position
                position['quantity'] -= quantity
                if position['quantity'] <= 0:
                    position['quantity'] = 0
                    position['avg_price'] = 0.0
                    position['entry_time'] = None
    
    def get_position(self, pair: str) -> Dict:
        """Get current position for pair"""
        return self.positions.get(pair, {
            'quantity': 0, 
            'avg_price': 0.0, 
            'entry_time': None,
            'max_profit_pips': 0.0,
            'max_loss_pips': 0.0
        })
    
    def get_advanced_performance_summary(self) -> Dict:
        """Get performance summary"""
        win_rate = (self.successful_trades / self.trades_count * 100) if self.trades_count > 0 else 0
        
        # Calculate additional metrics
        avg_trade_duration = 0
        avg_winning_trade = 0
        avg_losing_trade = 0
        
        if self.trade_history:
            # Trade duration
            durations = [t['duration_minutes'] for t in self.trade_history if t['duration_minutes']]
            avg_trade_duration = sum(durations) / len(durations) if durations else 0
            
            # Winning vs losing trade analysis
            winning_trades = [t['pnl_usd'] for t in self.trade_history if t['pnl_usd'] > 0]
            losing_trades = [t['pnl_usd'] for t in self.trade_history if t['pnl_usd'] < 0]
            
            avg_winning_trade = sum(winning_trades) / len(winning_trades) if winning_trades else 0
            avg_losing_trade = sum(losing_trades) / len(losing_trades) if losing_trades else 0
        
        # Calculate Sharpe ratio (simplified)
        sharpe_ratio = 0
        if self.trades_count > 5:
            trade_returns = [t['pnl_usd'] for t in self.trade_history]
            if trade_returns:
                avg_return = sum(trade_returns) / len(trade_returns)
                return_std = np.std(trade_returns) if len(trade_returns) > 1 else 1
                sharpe_ratio = (avg_return / return_std) if return_std > 0 else 0
        
        return {
            'total_trades': self.trades_count,
            'successful_trades': self.successful_trades,
            'win_rate': win_rate,
            'total_pnl': self.total_pnl,
            'max_drawdown': self.max_drawdown,
            'avg_trade_duration_minutes': avg_trade_duration,
            'avg_winning_trade': avg_winning_trade,
            'avg_losing_trade': avg_losing_trade,
            'sharpe_ratio': sharpe_ratio,
            'current_positions': self.positions,
            'market_regime': self.market_regime,
            'current_session': self.get_trading_session()
        }
    
    def get_trade_analysis(self) -> Dict:
        """Get detailed trade analysis"""
        if not self.trade_history:
            return {'message': 'No trades yet'}
        
        # Group trades by various criteria
        by_pair = {}
        by_session = {}
        by_regime = {}
        
        for trade in self.trade_history:
            # By pair
            pair = trade['pair']
            if pair not in by_pair:
                by_pair[pair] = {'trades': 0, 'pnl': 0, 'wins': 0}
            by_pair[pair]['trades'] += 1
            by_pair[pair]['pnl'] += trade['pnl_usd']
            if trade['pnl_usd'] > 0:
                by_pair[pair]['wins'] += 1
            
            # By session
            session = trade['session']
            if session not in by_session:
                by_session[session] = {'trades': 0, 'pnl': 0, 'wins': 0}
            by_session[session]['trades'] += 1
            by_session[session]['pnl'] += trade['pnl_usd']
            if trade['pnl_usd'] > 0:
                by_session[session]['wins'] += 1
            
            # By regime
            regime = trade['regime']
            if regime not in by_regime:
                by_regime[regime] = {'trades': 0, 'pnl': 0, 'wins': 0}
            by_regime[regime]['trades'] += 1
            by_regime[regime]['pnl'] += trade['pnl_usd']
            if trade['pnl_usd'] > 0:
                by_regime[regime]['wins'] += 1
       
        # Calculate win rates
        for category in [by_pair, by_session, by_regime]:
            for key, data in category.items():
                data['win_rate'] = (data['wins'] / data['trades'] * 100) if data['trades'] > 0 else 0
        
        return {
            'by_pair': by_pair,
            'by_session': by_session,
            'by_regime': by_regime,
            'total_trades': len(self.trade_history),
            'best_trade': max(self.trade_history, key=lambda x: x['pnl_usd']),
            'worst_trade': min(self.trade_history, key=lambda x: x['pnl_usd'])
        }

print("✓ Advanced Forex Day Trading Strategy created")

### Day Trading Bot

In [ ]:
class AdvancedForexDayTradingBot:
    """Advanced forex day trading bot with optimised profit strategies"""
    
    def __init__(self, environment: str = 'paper', pairs: List[str] = None, capital: float = 100):
        """Initialise advanced forex trading bot"""
        self.environment = environment
        self.pairs = pairs or ['EURUSD', 'GBPUSD']
        self.available_capital = capital
        self.logger = logging.getLogger(f'AdvancedForexBot-{environment}')
        
        # Initialise components
        self.ib_conn = CryptoIBConnection(environment)
        self.market_data = None
        self.order_manager = None
        self.strategy = None
        
        self.running = False
        self.last_equity_check = datetime.now()
        
        self.logger.info(f"Advanced forex bot initialised for {environment}")
        self.logger.info(f"Trading pairs: {', '.join(self.pairs)}")
        self.logger.info(f"Starting capital: ${capital:,}")
    
    def start(self) -> bool:
        """Start the advanced forex trading bot"""
        self.logger.info("Starting advanced forex trading bot...")
        
        if not self.ib_conn.connect():
            self.logger.error("Failed to connect to IB")
            return False
        
        # Initialise components
        self.market_data = ForexMarketData(self.ib_conn)
        self.order_manager = ForexOrderManager(self.ib_conn)
        self.strategy = AdvancedForexDayTradingStrategy()
        
        # Validate pairs
        available_pairs = self.market_data.get_available_pairs()
        working_pairs = [p for p in self.pairs if p in available_pairs]
        
        if working_pairs:
            self.pairs = working_pairs
            self.logger.info(f"Active trading pairs: {', '.join(self.pairs)}")
        else:
            self.logger.error("No valid trading pairs available")
            return False
        
        self.running = True
        self.logger.info("✓ Advanced forex trading bot started successfully")
        return True
    
    def stop(self):
        """Stop the advanced forex trading bot"""
        self.running = False
        if self.ib_conn:
            self.ib_conn.disconnect()
        self.logger.info("Advanced forex trading bot stopped")
    
    def run_advanced_trading_cycle(self):
        """Run advanced trading cycle with analysis"""
        if not self.running or not self.ib_conn.is_connected():
            return
        
        cycle_start = datetime.now()
        self.logger.info(f"Running advanced trading cycle...")
        
        # Update available capital based on current equity
        self.update_available_capital()
        
        for pair in self.pairs:
            try:
                # Get market data
                market_data = self.market_data.get_forex_price(pair)
                if not market_data:
                    continue
                
                # Get historical data for advanced analysis
                historical_df = self.market_data.get_historical_forex_data(pair, '2 D', '5 mins')
                if historical_df is None or historical_df.empty:
                    self.logger.warning(f"No historical data for {pair}")
                    continue
                
                # Calculate advanced indicators
                indicators = self.strategy.calculate_advanced_indicators(historical_df)
                
                # Advanced trading decisions
                if self.strategy.should_buy_forex(pair, market_data, indicators):
                    self.execute_advanced_buy(pair, market_data, indicators)
                elif self.strategy.should_sell_forex(pair, market_data, indicators):
                    self.execute_advanced_sell(pair, market_data, indicators)
                
                # Monitor existing positions
                self.monitor_position_risk(pair, market_data, indicators)
                
                time.sleep(0.3)  # Slight delay between pairs
                
            except Exception as e:
                self.logger.error(f"Error processing {pair}: {e}")
        
        cycle_duration = (datetime.now() - cycle_start).total_seconds()
        self.logger.info(f"Advanced trading cycle completed in {cycle_duration:.1f}s")
    
    def execute_advanced_buy(self, pair: str, market_data: Dict, indicators: Dict):
        """Execute advanced buy with dynamic position sizing"""
        try:
            price = market_data['last']
            
            # Calculate dynamic position size
            quantity = self.strategy.calculate_dynamic_position_size(
                pair, indicators, self.available_capital
            )
            
            # Calculate expected stops for logging
            stop_levels = self.strategy.calculate_dynamic_stops(pair, indicators, price)
            
            if self.order_manager.place_forex_market_order(pair, 'BUY', quantity):
                self.strategy.update_position(pair, 'BUY', quantity, price)
                
                self.logger.info(f"✓ ADVANCED BUY {pair}:")
                self.logger.info(f"  Quantity: {quantity:,} units")
                self.logger.info(f"  Entry: {price:.5f}")
                self.logger.info(f"  Stop Loss: {stop_levels['stop_loss_pips']:.1f} pips")
                self.logger.info(f"  Take Profit: {stop_levels['take_profit_pips']:.1f} pips")
                self.logger.info(f"  Risk:Reward = 1:{stop_levels['risk_reward_ratio']:.1f}")
                self.logger.info(f"  Session: {self.strategy.get_trading_session()}")
                self.logger.info(f"  Regime: {self.strategy.market_regime}")
            
        except Exception as e:
            self.logger.error(f"Error executing advanced buy for {pair}: {e}")
    
    def execute_advanced_sell(self, pair: str, market_data: Dict, indicators: Dict):
        """Execute advanced sell with detailed logging"""
        try:
            position = self.strategy.get_position(pair)
            if position['quantity'] <= 0:
                return
            
            price = market_data['last']
            quantity = position['quantity']
            entry_price = position['avg_price']
            
            # Calculate trade metrics
            pip_size = 0.0001 if 'JPY' not in pair else 0.01
            pnl_pips = (price - entry_price) / pip_size
            pnl_usd = (price - entry_price) * quantity
            
            # Calculate trade duration
            duration_minutes = 0
            if position['entry_time']:
                duration_minutes = (datetime.now() - position['entry_time']).total_seconds() / 60
            
            if self.order_manager.place_forex_market_order(pair, 'SELL', quantity):
                self.strategy.update_position(pair, 'SELL', quantity, price)
                
                self.logger.info(f"✓ ADVANCED SELL {pair}:")
                self.logger.info(f"  Quantity: {quantity:,} units")
                self.logger.info(f"  Entry: {entry_price:.5f}")
                self.logger.info(f"  Exit: {price:.5f}")
                self.logger.info(f"  P&L: {pnl_pips:+.1f} pips (${pnl_usd:+.2f})")
                self.logger.info(f"  Duration: {duration_minutes:.1f} minutes")
                self.logger.info(f"  Session: {self.strategy.get_trading_session()}")
            
        except Exception as e:
            self.logger.error(f"Error executing advanced sell for {pair}: {e}")
    
    def monitor_position_risk(self, pair: str, market_data: Dict, indicators: Dict):
        """Monitor and adjust existing positions for risk management"""
        position = self.strategy.get_position(pair)
        if position['quantity'] <= 0:
            return
        
        try:
            price = market_data['last']
            entry_price = position['avg_price']
            pip_size = 0.0001 if 'JPY' not in pair else 0.01
            current_pnl_pips = (price - entry_price) / pip_size
            
            # Update position tracking
            if current_pnl_pips > position['max_profit_pips']:
                position['max_profit_pips'] = current_pnl_pips
            if current_pnl_pips < position['max_loss_pips']:
                position['max_loss_pips'] = current_pnl_pips
            
            # Trailing stop logic
            if current_pnl_pips > 30:  # If profit > 30 pips
                trailing_stop_pips = current_pnl_pips * 0.5  # Trail at 50% of profit
                if abs(position['max_loss_pips']) > trailing_stop_pips:
                    self.logger.info(f"⚠ Trailing stop activated for {pair}: {trailing_stop_pips:.1f} pips")
            
        except Exception as e:
            self.logger.error(f"Error monitoring {pair} position: {e}")
    
    def update_available_capital(self):
        """Update available capital based on current equity"""
        try:
            # In production, this would query IB account value
            # For demo, calculate based on positions and P&L
            total_position_value = 0
            
            for pair, position in self.strategy.positions.items():
                if position['quantity'] > 0:
                    current_price = self.market_data.last_prices.get(pair, position['avg_price'])
                    position_value = position['quantity'] * current_price
                    total_position_value += position_value
            
            # Update capital (starting capital + P&L - current positions)
            self.available_capital = 100 + self.strategy.total_pnl - total_position_value
            
        except Exception as e:
            self.logger.error(f"Error updating capital: {e}")
    
    def get_advanced_status(self) -> Dict:
        """Get bot status"""
        status = {
            'running': self.running,
            'connected': self.ib_conn.is_connected() if self.ib_conn else False,
            'environment': self.environment,
            'pairs': self.pairs,
            'available_capital': self.available_capital,
            'current_session': self.strategy.get_trading_session() if self.strategy else 'unknown',
            'market_regime': self.strategy.market_regime if self.strategy else 'unknown'
        }
        
        if self.strategy:
            status['performance'] = self.strategy.get_advanced_performance_summary()
            status['trade_analysis'] = self.strategy.get_trade_analysis()
        
        return status
    
    def run_once(self):
        """Run advanced trading logic once"""
        if not self.running:
            self.logger.warning("Advanced bot not started")
            return
        
        self.run_advanced_trading_cycle()

print("✓ Advanced Forex Day Trading Bot created")

### Clean Up Existing Connections

In [ ]:
# Clean up any existing IB connections
def cleanup_ib_connections():
    """Clean up existing IB connections"""
    try:
        # Try to disconnect any existing instances
        ib_temp = IB()
        if ib_temp.isConnected():
            ib_temp.disconnect()
        time.sleep(2)
        print("✓ Cleaned up existing connections")
    except:
        print("✓ No existing connections to clean up")

# Run cleanup
cleanup_ib_connections()

### Day Trading Bot Execution

In [ ]:
def demo_forex_trading():
    """Demonstrate forex trading bot (corrected version)"""
    
    print("=== Advanced Forex Day Trading Bot Demo ===\n")
    
    # Clean up first
    cleanup_ib_connections()
    
    # Create bot using CORRECT class name
    bot = AdvancedForexDayTradingBot(
        environment='paper',
        pairs=['EURUSD', 'GBPUSD'],
        capital=100
    )
    
    try:
        if bot.start():
            print("Bot started successfully\n")
            
            # Test available pairs
            available_pairs = bot.market_data.get_available_pairs()
            
            if available_pairs:
                print(f"Available forex pairs: {', '.join(available_pairs)}")
                bot.pairs = available_pairs[:2]
                
                # Run one cycle
                bot.run_advanced_trading_cycle()
                
                # Show results
                status = bot.get_advanced_status()
                print(f"Environment: {status['environment']}")
                print(f"Connected: {status['connected']}")
                
                print("Demo completed successfully")
            else:
                print("No forex pairs available")
        else:
            print("Failed to start bot")
            
    except Exception as e:
        print(f"Demo error: {e}")
    
    finally:
        bot.stop()
        print("Bot stopped")

# Run the demo
demo_forex_trading()

### Advanced Trading Loop

In [ ]:
def run_forex_day_trading_session(duration_minutes: int = 60, 
                                 environment: str = 'paper',
                                 pairs: List[str] = None):
    """Run a forex day trading session"""
    
    pairs = pairs or ['EURUSD', 'GBPUSD', 'USDJPY']
    bot = AdvancedForexDayTradingBot(environment, pairs)
    
    print(f"Starting {duration_minutes}-minute forex trading session...")
    print(f"Environment: {environment.upper()}")
    print(f"Forex Pairs: {', '.join(pairs)}")
    
    if not bot.start():
        print("Failed to start trading session")
        return
    
    try:
        # Test which pairs are actually available
        available_pairs = bot.market_data.get_available_pairs()
        if not available_pairs:
            print("No forex pairs available")
            return
        
        # Use only available pairs
        bot.pairs = available_pairs[:3]  # Limit to 3 pairs for demo
        print(f"Active pairs: {', '.join(bot.pairs)}")
        
        start_time = datetime.now()
        cycle_count = 0
        
        print(f"\n{'='*50}")
        print(f"FOREX DAY TRADING SESSION STARTED")
        print(f"{'='*50}")
        
        while (datetime.now() - start_time).total_seconds() < duration_minutes * 60:
            cycle_count += 1
            current_time = datetime.now().strftime("%H:%M:%S")
            
            print(f"\n--- Trading Cycle {cycle_count} ({current_time}) ---")
            
            # Show current forex rates
            print("Current Rates:")
            for pair in bot.pairs:
                try:
                    price_data = bot.market_data.get_forex_price(pair)
                    if price_data:
                        rate = price_data['last']
                        spread = ""
                        if price_data['bid'] and price_data['ask']:
                            spread_pips = (price_data['ask'] - price_data['bid']) * 100
                            if 'JPY' in pair:
                                spread_pips = spread_pips / 100
                            spread = f" (Spread: {spread_pips:.1f} pips)"
                        print(f"  {pair}: {rate:.5f}{spread}")
                    else:
                        print(f"  {pair}: No data")
                except Exception as e:
                    print(f"  {pair}: Error - {e}")
            
            # Run trading logic
            bot.run_once()
            
            # Show performance
            status = bot.get_status()
            if 'performance' in status:
                perf = status['performance']
                if perf['total_trades'] > 0:
                    print(f"\nPerformance:")
                    print(f"  Trades: {perf['total_trades']}")
                    print(f"  Win Rate: {perf['win_rate']:.1f}%")
                    print(f"  P&L: ${perf['total_pnl']:.2f}")
                
                # Show current positions
                positions = perf['current_positions']
                active_positions = {k: v for k, v in positions.items() if v['quantity'] > 0}
                if active_positions:
                    print(f"  Active Positions:")
                    for pair, pos in active_positions.items():
                        pnl_pct = 0
                        if pos['avg_price'] > 0:
                            current_price = bot.market_data.last_prices.get(pair, pos['avg_price'])
                            pnl_pct = ((current_price - pos['avg_price']) / pos['avg_price']) * 100
                        
                        print(f"    {pair}: {pos['quantity']:,} units @ {pos['avg_price']:.5f} "
                              f"({pnl_pct:+.2f}%)")
                else:
                    print(f"  No active positions")
            
            # Calculate time remaining
            elapsed = (datetime.now() - start_time).total_seconds()
            remaining = (duration_minutes * 60) - elapsed
            
            if remaining > 30:
                print(f"\nNext cycle in 30 seconds... ({remaining/60:.1f} minutes remaining)")
                time.sleep(30)
            else:
                break
        
        print(f"\n{'='*50}")
        print(f"FOREX TRADING SESSION COMPLETE")
        print(f"{'='*50}")
        
        # Final results
        final_status = bot.get_status()
        if 'performance' in final_status:
            perf = final_status['performance']
            print(f"\nFinal Results:")
            print(f"  Session Duration: {duration_minutes} minutes")
            print(f"  Trading Cycles: {cycle_count}")
            print(f"  Total Trades: {perf['total_trades']}")
            if perf['total_trades'] > 0:
                print(f"  Successful Trades: {perf['successful_trades']}")
                print(f"  Win Rate: {perf['win_rate']:.1f}%")
                print(f"  Total P&L: ${perf['total_pnl']:.2f}")
                
                # Show final positions
                final_positions = perf['current_positions']
                active_final = {k: v for k, v in final_positions.items() if v['quantity'] > 0}
                if active_final:
                    print(f"  Open Positions:")
                    for pair, pos in active_final.items():
                        print(f"    {pair}: {pos['quantity']:,} units @ {pos['avg_price']:.5f}")
            else:
                print(f"  No trades executed")
    
    except KeyboardInterrupt:
        print("\n⚠ Trading session interrupted by user")
    except Exception as e:
        print(f"Trading session error: {e}")
    finally:
        bot.stop()
        print("\nTrading session ended")

def run_quick_forex_test(minutes: int = 2):
    """Quick 2-minute forex trading test"""
    print(f"=== {minutes}-Minute Forex Test ===")
    run_forex_day_trading_session(duration_minutes=minutes, pairs=['EURUSD'])

def run_full_forex_session(minutes: int = 30):
    """Full forex trading session"""
    print(f"=== {minutes}-Minute Forex Session ===")
    run_forex_day_trading_session(
        duration_minutes=minutes, 
        pairs=['EURUSD', 'GBPUSD', 'USDJPY']
    )

print("✓ Advanced forex trading loop created")
print("\nUsage:")
print("• run_quick_forex_test(2) - 2-minute test")
print("• run_full_forex_session(30) - 30-minute session")
print("• run_forex_day_trading_session(60) - Custom duration")

### Show Advanced Performance

In [ ]:
def show_advanced_performance(bot: AdvancedForexDayTradingBot):
    """Show detailed advanced performance metrics"""
    
    if not bot or not bot.strategy:
        print("No performance data available")
        return
    
    perf = bot.strategy.get_advanced_performance_summary()
    analysis = bot.strategy.get_trade_analysis()
    
    print(f"\n{'='*60}")
    print(f"ADVANCED FOREX TRADING PERFORMANCE")
    print(f"{'='*60}")
    print(f"Environment: {bot.environment.upper()}")
    print(f"Trading Pairs: {', '.join(bot.pairs)}")
    print(f"Available Capital: £{bot.available_capital:,.2f}")
    
    if perf['total_trades'] > 0:
        roi_pct = (perf['total_pnl'] / 100) * 100  # Assuming £100 starting capital
        
        print(f"\nTrading Statistics:")
        print(f"  Total Trades: {perf['total_trades']}")
        print(f"  Successful Trades: {perf['successful_trades']}")
        print(f"  Win Rate: {perf['win_rate']:.1f}%")
        print(f"  Total P&L: £{perf['total_pnl']:+.2f}")
        print(f"  Return on Investment: {roi_pct:+.2f}%")
        print(f"  Max Drawdown: £{perf['max_drawdown']:.2f}")
        print(f"  Average Trade Duration: {perf['avg_trade_duration_minutes']:.1f} minutes")
        print(f"  Sharpe Ratio: {perf['sharpe_ratio']:.2f}")
        
        if perf['avg_winning_trade'] > 0:
            print(f"  Average Winning Trade: £{perf['avg_winning_trade']:.2f}")
        if perf['avg_losing_trade'] < 0:
            print(f"  Average Losing Trade: £{perf['avg_losing_trade']:.2f}")
        
        # Risk metrics
        profit_factor = abs(perf['avg_winning_trade'] / perf['avg_losing_trade']) if perf['avg_losing_trade'] < 0 else 0
        print(f"  Profit Factor: {profit_factor:.2f}")
        
        # Show current positions
        active_positions = {k: v for k, v in perf['current_positions'].items() if v['quantity'] > 0}
        if active_positions:
            print(f"\nActive Positions:")
            total_exposure = 0
            for pair, pos in active_positions.items():
                current_rate = bot.market_data.last_prices.get(pair, pos['avg_price'])
                pip_size = 0.0001 if 'JPY' not in pair else 0.01
                pnl_pips = (current_rate - pos['avg_price']) / pip_size
                position_value = pos['quantity'] * current_rate
                total_exposure += position_value
                
                duration = "Unknown"
                if pos['entry_time']:
                    duration_mins = (datetime.now() - pos['entry_time']).total_seconds() / 60
                    duration = f"{duration_mins:.0f}m"
                
                print(f"    {pair}: {pos['quantity']:,} units @ {pos['avg_price']:.5f}")
                print(f"      Current: {current_rate:.5f} | P&L: {pnl_pips:+.1f} pips | Duration: {duration}")
                print(f"      Position Value: £{position_value:,.2f}")
            
            exposure_pct = (total_exposure / bot.available_capital) * 100 if bot.available_capital > 0 else 0
            print(f"\nTotal Exposure: £{total_exposure:,.2f} ({exposure_pct:.1f}% of capital)")
        else:
            print(f"\nNo active positions")
        
        # Performance by trading session
        if analysis and 'by_session' in analysis:
            print(f"\nPerformance by Trading Session:")
            for session, data in analysis['by_session'].items():
                if data['trades'] > 0:
                    print(f"  {session.capitalize()}: {data['trades']} trades, £{data['pnl']:+.2f}, {data['win_rate']:.0f}% wins")
        
        # Performance by market regime
        if analysis and 'by_regime' in analysis:
            print(f"\nPerformance by Market Regime:")
            for regime, data in analysis['by_regime'].items():
                if data['trades'] > 0:
                    print(f"  {regime.capitalize()}: {data['trades']} trades, £{data['pnl']:+.2f}, {data['win_rate']:.0f}% wins")
        
        # Best and worst trades
        if analysis and 'best_trade' in analysis and 'worst_trade' in analysis:
            best = analysis['best_trade']
            worst = analysis['worst_trade']
            print(f"\nTrade Extremes:")
            print(f"  Best Trade: {best['pair']} - {best['pnl_pips']:+.1f} pips (£{best['pnl_usd']:+.2f})")
            print(f"  Worst Trade: {worst['pair']} - {worst['pnl_pips']:+.1f} pips (£{worst['pnl_usd']:+.2f})")
    else:
        print(f"\nNo trades executed yet")
        print(f"Current Market Conditions:")
        print(f"  Session: {perf['current_session'].upper()}")
        print(f"  Regime: {perf['market_regime'].upper()}")
    
    print(f"{'='*60}")

### Quick Test Functions

In [ ]:
def quick_forex_price_check(environment: str = 'paper', pairs: List[str] = None):
    """Quick function to check forex rates"""
    
    pairs = pairs or ['EURUSD', 'GBPUSD', 'USDJPY']
    
    print(f"=== Forex Rate Check ({environment.upper()}) ===")
    
    ib_conn = CryptoIBConnection(environment)
    
    if not ib_conn.connect():
        print("Failed to connect to IB")
        return
    
    try:
        market_data = ForexMarketData(ib_conn)
        
        print(f"\nCurrent Forex Rates:")
        for pair in pairs:
            price_data = market_data.get_forex_price(pair)
            if price_data:
                rate = price_data['last']
                spread_info = ""
                
                if price_data['bid'] and price_data['ask']:
                    spread = price_data['ask'] - price_data['bid']
                    spread_pips = spread * 100
                    if 'JPY' in pair:
                        spread_pips = spread_pips / 100
                    spread_info = f" | Spread: {spread_pips:.1f} pips"
                
                print(f"  {pair}: {rate:.5f}{spread_info}")
            else:
                print(f"  {pair}: No data available")
            
            time.sleep(1)
    
    except Exception as e:
        print(f"Rate check error: {e}")
    finally:
        ib_conn.disconnect()
        print("\nRate check completed")

def test_forex_indicators(pair: str = 'EURUSD', environment: str = 'paper'):
    """Test technical indicators for a forex pair"""
    
    print(f"=== Testing Indicators for {pair} ===")
    
    ib_conn = CryptoIBConnection(environment)
    
    if not ib_conn.connect():
        print("Failed to connect to IB")
        return
    
    try:
        market_data = ForexMarketData(ib_conn)
        strategy = ForexDayTradingStrategy()
        
        # Get historical data
        df = market_data.get_historical_forex_data(pair, '1 D', '5 mins')
        
        if df is not None and not df.empty:
            print(f"\nHistorical data: {len(df)} bars")
            print(f"Rate range: {df['low'].min():.5f} - {df['high'].max():.5f}")
            
            # Calculate indicators
            indicators = strategy.calculate_forex_indicators(df)
            
            print(f"\nTechnical Indicators:")
            for key, value in indicators.items():
                if isinstance(value, float):
                    if 'sma' in key.lower():
                        print(f"  {key}: {value:.5f}")
                    elif 'pip' in key.lower():
                        print(f"  {key}: {value:.1f}")
                    else:
                        print(f"  {key}: {value:.2f}")
                else:
                    print(f"  {key}: {value}")
            
            # Test current signals
            current_data = market_data.get_forex_price(pair)
            if current_data:
                print(f"\nCurrent Rate: {current_data['last']:.5f}")
                
                buy_signal = strategy.should_buy_forex(pair, current_data, indicators)
                sell_signal = strategy.should_sell_forex(pair, current_data, indicators)
                
                print(f"Buy Signal: {'🟢 YES' if buy_signal else '🔴 NO'}")
                print(f"Sell Signal: {'🟢 YES' if sell_signal else '🔴 NO'}")
        
        else:
            print("No historical data available")
    
    except Exception as e:
        print(f"Indicator test error: {e}")
    finally:
        ib_conn.disconnect()
        print("\nIndicator test completed")

print("=== Quick Test Functions Ready ===")
print("• quick_forex_price_check() - Check current rates")
print("• test_forex_indicators('EURUSD') - Test indicators")
print("• run_quick_forex_test(2) - 2-minute trading test")

### Monitor and Control

In [ ]:
def monitor_forex_trading(bot: AdvancedForexDayTradingBot, cycles: int = 10):
    """Monitor forex trading bot for specified cycles"""
    
    if not bot.running:
        print("Bot not running")
        return
    
    print(f"Monitoring forex trading for {cycles} cycles...")
    
    for i in range(cycles):
        print(f"\n--- Cycle {i+1}/{cycles} ({datetime.now().strftime('%H:%M:%S')}) ---")
        
        try:
            # Show current rates first
            print("Current Forex Rates:")
            for pair in bot.pairs:
                price_data = bot.market_data.get_forex_price(pair)
                if price_data:
                    rate = price_data['last']
                    change = ""
                    if pair in bot.market_data.last_prices:
                        prev_rate = bot.market_data.last_prices[pair]
                        change_pips = (rate - prev_rate) * 100
                        if 'JPY' in pair:
                            change_pips = change_pips / 100
                        change = f" ({change_pips:+.1f} pips)"
                    print(f"  {pair}: {rate:.5f}{change}")
                else:
                    print(f"  {pair}: No data")
            
            # Run trading cycle
            bot.run_once()
            
            # Show performance
            status = bot.get_status()
            if 'performance' in status:
                perf = status['performance']
                if perf['total_trades'] > 0:
                    print(f"\nPerformance Update:")
                    print(f"  Total Trades: {perf['total_trades']}")
                    print(f"  Win Rate: {perf['win_rate']:.1f}%")
                    print(f"  P&L: ${perf['total_pnl']:.2f}")
                
                # Show position details
                positions = perf['current_positions']
                active_positions = {k: v for k, v in positions.items() if v['quantity'] > 0}
                if active_positions:
                    print(f"  Open Positions:")
                    for pair, pos in active_positions.items():
                        current_rate = bot.market_data.last_prices.get(pair, pos['avg_price'])
                        pnl_pips = (current_rate - pos['avg_price']) * 100
                        if 'JPY' in pair:
                            pnl_pips = pnl_pips / 100
                        
                        print(f"    {pair}: {pos['quantity']:,} units @ {pos['avg_price']:.5f} "
                              f"(P&L: {pnl_pips:+.1f} pips)")
                else:
                    print(f"  No open positions")
        
        except Exception as e:
            print(f"Monitoring error: {e}")
        
        # Wait before next cycle
        if i < cycles - 1:
            print("Waiting 30 seconds for next cycle...")
            time.sleep(30)
    
    print(f"\n✓ Monitoring completed after {cycles} cycles")

def emergency_close_all_forex_positions(bot: AdvancedForexDayTradingBot):
    """Emergency function to close all forex positions"""
    
    if not bot.running:
        print("Bot not running")
        return
    
    print("⚠ EMERGENCY STOP: Closing all forex positions...")
    
    try:
        closed_positions = 0
        for pair in bot.pairs:
            position = bot.strategy.get_position(pair)
            if position['quantity'] > 0:
                market_data = bot.market_data.get_forex_price(pair)
                if market_data:
                    print(f"Closing {pair}: {position['quantity']:,} units")
                    bot.execute_sell(pair, market_data)
                    closed_positions += 1
        
        if closed_positions > 0:
            print(f"✓ Closed {closed_positions} positions")
        else:
            print("No positions to close")
        
    except Exception as e:
        print(f"Emergency stop error: {e}")

def show_forex_performance(bot: AdvancedForexDayTradingBot):
    """Show detailed forex trading performance"""
    
    if not bot.strategy:
        print("No strategy data available")
        return
    
    perf = bot.strategy.get_performance_summary()
    
    print(f"\n{'='*40}")
    print(f"FOREX TRADING PERFORMANCE")
    print(f"{'='*40}")
    print(f"Environment: {bot.environment.upper()}")
    print(f"Trading Pairs: {', '.join(bot.pairs)}")
    print(f"Available Capital: ${bot.available_capital:,}")
    
    if perf['total_trades'] > 0:
        print(f"\nTrading Statistics:")
        print(f"  Total Trades: {perf['total_trades']}")
        print(f"  Successful Trades: {perf['successful_trades']}")
        print(f"  Win Rate: {perf['win_rate']:.1f}%")
        print(f"  Total P&L: ${perf['total_pnl']:.2f}")
        print(f"  Average P&L per Trade: ${perf['total_pnl']/perf['total_trades']:.2f}")
        
        # Calculate return percentage
        roi_pct = (perf['total_pnl'] / bot.available_capital) * 100
        print(f"  Return on Capital: {roi_pct:.2f}%")
    else:
        print(f"\nNo trades executed yet")
    
    # Show current positions
    positions = perf['current_positions']
    active_positions = {k: v for k, v in positions.items() if v['quantity'] > 0}
    
    if active_positions:
        print(f"\nCurrent Positions:")
        total_position_value = 0
        for pair, pos in active_positions.items():
            current_rate = bot.market_data.last_prices.get(pair, pos['avg_price'])
            position_value = pos['quantity'] * current_rate
            pnl_pips = (current_rate - pos['avg_price']) * 100
            if 'JPY' in pair:
                pnl_pips = pnl_pips / 100
            
            print(f"  {pair}:")
            print(f"    Quantity: {pos['quantity']:,} units")
            print(f"    Entry Rate: {pos['avg_price']:.5f}")
            print(f"    Current Rate: {current_rate:.5f}")
            print(f"    P&L: {pnl_pips:+.1f} pips")
            print(f"    Position Value: ${position_value:,.2f}")
            
            total_position_value += position_value
        
        print(f"\nTotal Position Value: ${total_position_value:,.2f}")
        exposure_pct = (total_position_value / bot.available_capital) * 100
        print(f"Capital Exposure: {exposure_pct:.1f}%")
    else:
        print(f"\nNo open positions")
    
    print(f"{'='*40}")

# Enhanced trading functions
def run_intensive_forex_session(duration_minutes: int = 15, 
                               cycle_frequency_seconds: int = 20):
    """Run intensive forex trading with frequent cycles"""
    
    pairs = ['EURUSD', 'GBPUSD']  # Focus on major pairs
    bot = AdvancedForexDayTradingBot('paper', pairs)
    
    print(f"=== INTENSIVE FOREX SESSION ===")
    print(f"Duration: {duration_minutes} minutes")
    print(f"Cycle Frequency: {cycle_frequency_seconds} seconds")
    
    if not bot.start():
        print("Failed to start session")
        return
    
    try:
        start_time = datetime.now()
        cycle_count = 0
        
        while (datetime.now() - start_time).total_seconds() < duration_minutes * 60:
            cycle_count += 1
            
            # Run trading cycle
            bot.run_trading_cycle()
            
            # Quick status update every 5 cycles
            if cycle_count % 5 == 0:
                print(f"\nCycle {cycle_count} Status:")
                show_forex_performance(bot)
            
            # Wait for next cycle
            remaining_time = (duration_minutes * 60) - (datetime.now() - start_time).total_seconds()
            if remaining_time > cycle_frequency_seconds:
                time.sleep(cycle_frequency_seconds)
            else:
                break
        
        print(f"\n=== SESSION COMPLETE ===")
        print(f"Total Cycles: {cycle_count}")
        show_forex_performance(bot)
        
    except KeyboardInterrupt:
        print("\nSession interrupted by user")
    except Exception as e:
        print(f"Session error: {e}")
    finally:
        bot.stop()

def run_forex_scalping_session(duration_minutes: int = 10):
    """Run high-frequency forex scalping session"""
    
    bot = AdvancedForexDayTradingBot('paper', ['EURUSD'])  # Single pair for scalping
    
    print(f"=== FOREX SCALPING SESSION ===")
    print(f"Duration: {duration_minutes} minutes")
    print("Strategy: High-frequency scalping on EURUSD")
    
    if not bot.start():
        print("Failed to start scalping session")
        return
    
    try:
        # Adjust strategy for scalping
        bot.strategy.take_profit_pips = 10  # Smaller profit target
        bot.strategy.stop_loss_pips = 5     # Tighter stop loss
        bot.strategy.momentum_threshold = 0.05  # More sensitive
        
        start_time = datetime.now()
        cycle_count = 0
        
        print("Starting scalping...")
        
        while (datetime.now() - start_time).total_seconds() < duration_minutes * 60:
            cycle_count += 1
            
            # Fast trading cycles
            bot.run_trading_cycle()
            
            # Quick update every 10 cycles
            if cycle_count % 10 == 0:
                status = bot.get_status()
                if 'performance' in status:
                    perf = status['performance']
                    print(f"Cycle {cycle_count}: {perf['total_trades']} trades, "
                          f"${perf['total_pnl']:.2f} P&L")
            
            # Short wait for scalping
            time.sleep(10)
        
        print(f"\n=== SCALPING SESSION COMPLETE ===")
        show_forex_performance(bot)
        
    except Exception as e:
        print(f"Scalping error: {e}")
    finally:
        bot.stop()

print("✓ Advanced forex trading loops created")
print("\nAvailable Functions:")
print("• run_forex_day_trading_session(60) - Standard session")
print("• run_intensive_forex_session(15) - High-frequency trading")
print("• run_forex_scalping_session(10) - Scalping strategy")
print("• monitor_forex_trading(bot, 10) - Monitor existing bot")
print("• emergency_close_all_forex_positions(bot) - Emergency stop")
print("• show_forex_performance(bot) - Detailed performance")

### Simple Forex Trading

In [ ]:
# Simple usage example for advanced forex trading
print("=== Advanced Forex Day Trading Bot Usage ===\n")

# Step 1: Create advanced forex bot
bot = AdvancedForexDayTradingBot(
    environment='paper',  # Use 'paper' for testing, 'live' for real trading
    pairs=['EURUSD', 'GBPUSD', 'USDJPY'],  # Major forex pairs
    capital=100  # Starting capital
)

# Step 2: Start advanced bot
if bot.start():
    print("✓ Advanced bot started successfully!")
    
    # Step 3: Show trading session and market regime
    session = bot.strategy.get_trading_session()
    regime = bot.strategy.market_regime
    print(f"\nMarket Conditions:")
    print(f"  Trading Session: {session.upper()}")
    print(f"  Market Regime: {regime.upper()}")
    
    # Step 4: Check current rates with advanced analysis
    print(f"\nAdvanced Rate Analysis:")
    for pair in bot.pairs:
        price_data = bot.market_data.get_forex_price(pair)
        if price_data:
            rate = price_data['last']
            print(f"  {pair}: {rate:.5f}")
            
            # Get historical data for quick analysis
            hist_data = bot.market_data.get_historical_forex_data(pair, '1 D', '15 mins')
            if hist_data is not None:
                indicators = bot.strategy.calculate_advanced_indicators(hist_data)
                
                # Show key indicators
                if 'rsi_14' in indicators:
                    rsi = indicators['rsi_14']
                    rsi_status = "Oversold" if rsi < 30 else "Overbought" if rsi > 70 else "Neutral"
                    print(f"    RSI: {rsi:.1f} ({rsi_status})")
                
                if 'trend_direction' in indicators:
                    trend = indicators['trend_direction']
                    strength = indicators.get('trend_strength', 0)
                    print(f"    Trend: {trend.capitalize()} (Strength: {strength:.2f}%)")
        else:
            print(f"  {pair}: No data available")
    
    # Step 5: Run advanced trading cycle
    print(f"\nRunning advanced trading cycle...")
    bot.run_once()
    
    # Step 6: Show status
    print(f"\nAdvanced Bot Status:")
    status = bot.get_advanced_status()
    
    print(f"  Environment: {status['environment'].upper()}")
    print(f"  Connected: {status['connected']}")
    print(f"  Active Pairs: {', '.join(status['pairs'])}")
    print(f"  Available Capital: ${status['available_capital']:,.2f}")
    print(f"  Current Session: {status['current_session'].upper()}")
    print(f"  Market Regime: {status['market_regime'].upper()}")
    
    # Step 7: Show performance metrics
    if 'performance' in status and status['performance']['total_trades'] > 0:
        perf = status['performance']
        print(f"\nPerformance Metrics:")
        print(f"  Total Trades: {perf['total_trades']}")
        print(f"  Win Rate: {perf['win_rate']:.1f}%")
        print(f"  Total P&L: ${perf['total_pnl']:+.2f}")
        print(f"  Max Drawdown: ${perf['max_drawdown']:.2f}")
        print(f"  Avg Trade Duration: {perf['avg_trade_duration_minutes']:.1f} minutes")
        print(f"  Sharpe Ratio: {perf['sharpe_ratio']:.2f}")
        
        if perf['avg_winning_trade'] > 0:
            print(f"  Avg Winning Trade: ${perf['avg_winning_trade']:.2f}")
        if perf['avg_losing_trade'] < 0:
            print(f"  Avg Losing Trade: ${perf['avg_losing_trade']:.2f}")
        
        # Show current positions
        active_positions = {k: v for k, v in perf['current_positions'].items() 
                           if v['quantity'] > 0}
        if active_positions:
            print(f"\nActive Positions:")
            for pair, pos in active_positions.items():
                current_rate = bot.market_data.last_prices.get(pair, pos['avg_price'])
                pip_size = 0.0001 if 'JPY' not in pair else 0.01
                pnl_pips = (current_rate - pos['avg_price']) / pip_size
                duration = "Unknown"
                if pos['entry_time']:
                    duration_mins = (datetime.now() - pos['entry_time']).total_seconds() / 60
                    duration = f"{duration_mins:.0f}m"
                
                print(f"    {pair}: {pos['quantity']:,} units @ {pos['avg_price']:.5f}")
                print(f"      Current: {current_rate:.5f} | P&L: {pnl_pips:+.1f} pips | Duration: {duration}")
        else:
            print(f"\nNo active positions")
        
        # Show trade analysis if available
        if 'trade_analysis' in status and status['trade_analysis'].get('total_trades', 0) > 0:
            analysis = status['trade_analysis']
            print(f"\nTrade Analysis:")
            
            # Best performing pair
            if 'by_pair' in analysis:
                best_pair = max(analysis['by_pair'].items(), key=lambda x: x[1]['pnl'])
                print(f"  Best Pair: {best_pair[0]} (${best_pair[1]['pnl']:+.2f}, {best_pair[1]['win_rate']:.0f}% wins)")
            
            # Best session
            if 'by_session' in analysis:
                best_session = max(analysis['by_session'].items(), key=lambda x: x[1]['pnl'])
                print(f"  Best Session: {best_session[0].upper()} (${best_session[1]['pnl']:+.2f})")
            
            # Best trade
            if 'best_trade' in analysis:
                best = analysis['best_trade']
                print(f"  Best Trade: {best['pair']} ({best['pnl_pips']:+.1f} pips, ${best['pnl_usd']:+.2f})")
    
    else:
        print(f"\nNo trades executed yet")
    
    # Step 8: Stop bot
    bot.stop()
    print(f"\n✓ Advanced demo completed")

else:
    print("✗ Failed to start advanced bot - check IB connection")

print("\n" + "="*70)
print("ADVANCED FOREX TRADING BOT - PROFIT OPTIMISED")
print("="*70)
print("Features:")
print("• Multi-timeframe technical analysis")
print("• Dynamic position sizing based on volatility")
print("• Session-aware trading (Asian/London/NY)")
print("• Market regime detection (Trending/Ranging/Volatile)")
print("• Advanced risk management with trailing stops")
print("• Performance analytics")
print("• Real-time equity and drawdown monitoring")
print("\nRisk Management:")
print("• Dynamic stop loss: 15-45 pips (ATR-based)")
print("• Take profit: 30-180 pips (regime-adjusted)")
print("• Maximum 25% capital exposure")
print("• Trailing stops on profitable positions")
print("\nQuick Commands:")
print("• run_advanced_forex_session(30) - 30-minute advanced session")
print("• show_advanced_performance(bot) - Detailed analytics")
print("• run_profit_optimised_session() - Full profit-focused session")
print("="*70)

### Production-Ready Forex Bot

In [ ]:
def create_advanced_production_forex_bot(environment: str = 'paper', 
                                        pairs: List[str] = None,
                                        capital: float = 100,
                                        risk_level: str = 'moderate') -> AdvancedForexDayTradingBot:
    """Create a production-ready advanced forex trading bot"""
    
    pairs = pairs or ['EURUSD', 'GBPUSD', 'USDJPY', 'NZDUSD']
    
    print(f"=== Creating Advanced Production Forex Bot ===")
    print(f"Environment: {environment.upper()}")
    print(f"Capital: ${capital:,}")
    print(f"Risk Level: {risk_level.upper()}")
    print(f"Target Pairs: {', '.join(pairs)}")
    
    # Create advanced bot
    bot = AdvancedForexDayTradingBot(environment, pairs, capital)
    
    # Adjust risk parameters based on risk level
    if bot.strategy:
        if risk_level.lower() == 'conservative':
            bot.strategy.base_position_pct = 0.05  # 5% positions
            bot.strategy.max_position_pct = 0.15   # 15% max exposure
            bot.strategy.base_stop_pips = 20       # Wider stops
            bot.strategy.base_profit_pips = 40     # Conservative profits
            print("📊 Risk Level: CONSERVATIVE")
            
        elif risk_level.lower() == 'aggressive':
            bot.strategy.base_position_pct = 0.12  # 12% positions
            bot.strategy.max_position_pct = 0.35   # 35% max exposure
            bot.strategy.base_stop_pips = 12       # Tighter stops
            bot.strategy.base_profit_pips = 25     # Faster profits
            print("🚀 Risk Level: AGGRESSIVE")
            
        else:  # moderate
            bot.strategy.base_position_pct = 0.08  # 8% positions
            bot.strategy.max_position_pct = 0.25   # 25% max exposure
            bot.strategy.base_stop_pips = 15       # Standard stops
            bot.strategy.base_profit_pips = 30     # Balanced profits
            print("⚖️ Risk Level: MODERATE")
    
    # Start and validate
    if bot.start():
        print("✓ Advanced bot started successfully")
        
        # Validate pairs and market conditions
        available = bot.market_data.get_available_pairs()
        working_pairs = [p for p in pairs if p in available]
        
        if working_pairs:
            bot.pairs = working_pairs
            print(f"✓ Active pairs: {', '.join(working_pairs)}")
            
            # Test market data and calculate initial indicators
            print("✓ Validating market conditions...")
            market_conditions = {}
            
            for pair in working_pairs:
                price_data = bot.market_data.get_forex_price(pair)
                if price_data:
                    # Get quick indicator reading
                    hist_data = bot.market_data.get_historical_forex_data(pair, '1 D', '15 mins')
                    if hist_data is not None:
                        indicators = bot.strategy.calculate_advanced_indicators(hist_data)
                        regime = bot.strategy.detect_market_regime(hist_data, indicators)
                        
                        market_conditions[pair] = {
                            'price': price_data['last'],
                            'regime': regime,
                            'rsi': indicators.get('rsi_14', 50),
                            'trend': indicators.get('trend_direction', 'neutral')
                        }
                        
                        print(f"  {pair}: {price_data['last']:.5f} | {regime} | {indicators.get('trend_direction', 'neutral')} ✓")
                    else:
                        print(f"  {pair}: Price OK, no historical data ⚠")
                else:
                    print(f"  {pair}: No data ✗")
            
            # Show session and regime info
            session = bot.strategy.get_trading_session()
            overall_regime = bot.strategy.market_regime
            
            print(f"\n📈 Market Analysis:")
            print(f"  Current Session: {session.upper()}")
            print(f"  Overall Regime: {overall_regime.upper()}")
            print(f"  Session Multiplier: {bot.strategy.session_weights.get(session, 1.0)}x")
            
            # Show expected performance parameters
            print(f"\n⚙️ Strategy Parameters:")
            print(f"  Base Position Size: {bot.strategy.base_position_pct*100:.0f}% of capital")
            print(f"  Max Exposure: {bot.strategy.max_position_pct*100:.0f}% of capital")
            print(f"  Base Stop Loss: {bot.strategy.base_stop_pips} pips")
            print(f"  Base Take Profit: {bot.strategy.base_profit_pips} pips")
            print(f"  Dynamic Stops: {'Enabled' if bot.strategy.dynamic_stop_loss else 'Disabled'}")
            
            print("✓ Advanced production bot ready")
            return bot
        else:
            print("✗ No working pairs available")
            bot.stop()
            return None
    else:
        print("✗ Failed to start advanced bot")
        return None

def run_production_advanced_session(duration_hours: float = 1.0, 
                                   risk_level: str = 'moderate',
                                   auto_adjust: bool = True):
    """Run advanced production forex trading session"""
    
    duration_minutes = int(duration_hours * 60)
    
    print(f"=== ADVANCED PRODUCTION SESSION ===")
    print(f"Duration: {duration_hours} hours ({duration_minutes} minutes)")
    print(f"Risk Profile: {risk_level.upper()}")
    print(f"Auto-Adjustment: {'Enabled' if auto_adjust else 'Disabled'}")
    
    # Create production bot with risk profile
    bot = create_advanced_production_forex_bot(
        environment='paper',  # Change to 'live' for real trading
        pairs=['EURUSD', 'GBPUSD', 'USDJPY', 'NZDUSD', 'AUDUSD'],
        capital=100,
        risk_level=risk_level
    )
    
    if not bot:
        return
    
    try:
        start_time = datetime.now()
        cycle = 0
        last_regime_check = start_time
        last_performance_check = start_time
        
        print(f"\n🚀 PRODUCTION SESSION STARTED")
        print(f"Start Time: {start_time.strftime('%H:%M:%S')}")
        
        while (datetime.now() - start_time).total_seconds() < duration_minutes * 60:
            cycle += 1
            current_time = datetime.now()
            
            # Run advanced trading cycle
            bot.run_advanced_trading_cycle()
            
            # Auto-adjust strategy every 15 minutes
            if auto_adjust and (current_time - last_regime_check).total_seconds() > 900:
                print(f"\n🔄 Auto-adjusting strategy parameters...")
                
                # Check if market regime changed
                old_regime = bot.strategy.market_regime
                
                # Re-evaluate regime based on recent performance
                if bot.strategy.trades_count > 3:
                    recent_trades = bot.strategy.trade_history[-3:]
                    recent_pnl = sum(t['pnl_usd'] for t in recent_trades)
                    
                    if recent_pnl < -50:  # Poor recent performance
                        print("  📉 Poor performance detected - reducing position sizes")
                        bot.strategy.base_position_pct *= 0.8
                    elif recent_pnl > 100:  # Excellent performance
                        print("  📈 Excellent performance - slightly increasing positions")
                        bot.strategy.base_position_pct *= 1.1
                        bot.strategy.base_position_pct = min(bot.strategy.base_position_pct, 0.15)
                
                print(f"  Regime: {old_regime} → {bot.strategy.market_regime}")
                last_regime_check = current_time
            
            # Performance update every 10 cycles
            if cycle % 10 == 0:
                elapsed_hours = (current_time - start_time).total_seconds() / 3600
                print(f"\n📊 Production Update (Cycle {cycle}, {elapsed_hours:.1f}h elapsed):")
                
                perf = bot.strategy.get_advanced_performance_summary()
                if perf['total_trades'] > 0:
                    hourly_pnl = perf['total_pnl'] / elapsed_hours if elapsed_hours > 0 else 0
                    print(f"  P&L: ${perf['total_pnl']:+.2f} (${hourly_pnl:+.1f}/hour)")
                    print(f"  Trades: {perf['total_trades']} | Win Rate: {perf['win_rate']:.0f}%")
                    print(f"  Drawdown: ${perf['max_drawdown']:.2f} | Sharpe: {perf['sharpe_ratio']:.2f}")
                    
                    # Show top performing pair
                    analysis = bot.strategy.get_trade_analysis()
                    if 'by_pair' in analysis:
                        best_pair = max(analysis['by_pair'].items(), key=lambda x: x[1]['pnl'])
                        print(f"  Best Pair: {best_pair[0]} (${best_pair[1]['pnl']:+.2f})")
                else:
                    print(f"  No trades executed yet")
                
                last_performance_check = current_time
            
            # Dynamic cycle timing based on performance and regime
            if bot.strategy.market_regime == 'volatile':
                cycle_wait = 25  # Faster in volatile markets
            elif bot.strategy.market_regime == 'trending':
                cycle_wait = 40  # Standard timing for trends
            else:  # ranging
                cycle_wait = 60  # Slower in ranging markets
            
            # Adjust timing based on recent performance
            if bot.strategy.trades_count > 0:
                win_rate = (bot.strategy.successful_trades / bot.strategy.trades_count) * 100
                if win_rate > 70:  # High win rate - trade more frequently
                    cycle_wait = int(cycle_wait * 0.8)
                elif win_rate < 40:  # Low win rate - slow down
                    cycle_wait = int(cycle_wait * 1.3)
            
            # Wait for next cycle
            remaining = (duration_minutes * 60) - (datetime.now() - start_time).total_seconds()
            if remaining > cycle_wait:
                time.sleep(cycle_wait)
            else:
                break
        
        # Final session summary
        end_time = datetime.now()
        session_duration = (end_time - start_time).total_seconds() / 3600
        
        print(f"\n🏁 ADVANCED PRODUCTION SESSION COMPLETE")
        print(f"{'='*60}")
        print(f"Session Duration: {session_duration:.2f} hours")
        print(f"Total Cycles: {cycle}")
        
        # Final results
        final_perf = bot.strategy.get_advanced_performance_summary()
        final_analysis = bot.strategy.get_trade_analysis()
        
        if final_perf['total_trades'] > 0:
            roi_pct = (final_perf['total_pnl'] / capital) * 100
            hourly_return = roi_pct / session_duration if session_duration > 0 else 0
            
            print(f"\n💰 Final Performance:")
            print(f"  Starting Capital: ${capital:,}")
            print(f"  Final P&L: ${final_perf['total_pnl']:+.2f}")
            print(f"  ROI: {roi_pct:+.2f}%")
            print(f"  Hourly Return: {hourly_return:+.2f}%/hour")
            print(f"  Total Trades: {final_perf['total_trades']}")
            print(f"  Win Rate: {final_perf['win_rate']:.1f}%")
            print(f"  Max Drawdown: ${final_perf['max_drawdown']:.2f}")
            print(f"  Sharpe Ratio: {final_perf['sharpe_ratio']:.2f}")
            
            # Show session performance breakdown
            if 'by_session' in final_analysis:
                print(f"\n📅 Performance by Session:")
                for session, data in final_analysis['by_session'].items():
                    if data['trades'] > 0:
                        print(f"  {session.capitalize()}: {data['trades']} trades, "
                              f"${data['pnl']:+.2f}, {data['win_rate']:.0f}% wins")
            
            # Show regime performance
            if 'by_regime' in final_analysis:
                print(f"\n🌊 Performance by Market Regime:")
                for regime, data in final_analysis['by_regime'].items():
                    if data['trades'] > 0:
                        print(f"  {regime.capitalize()}: {data['trades']} trades, "
                              f"${data['pnl']:+.2f}, {data['win_rate']:.0f}% wins")
            
            # Performance rating
            if roi_pct > 2.0:
                rating = "🌟 EXCELLENT"
            elif roi_pct > 1.0:
                rating = "✅ GOOD"
            elif roi_pct > 0:
                rating = "📈 POSITIVE"
            elif roi_pct > -1.0:
                rating = "⚠️ MINOR LOSS"
            else:
                rating = "🔴 SIGNIFICANT LOSS"
            
            print(f"\n🎯 Session Rating: {rating}")
            
        else:
            print(f"\n📊 No trades executed during session")
            print(f"Possible reasons:")
            print(f"  • Market conditions didn't meet entry criteria")
            print(f"  • Low volatility during session")
            print(f"  • Conservative risk parameters")
        
    except KeyboardInterrupt:
        print("\n⚠ Production session interrupted by user")
    except Exception as e:
        print(f"Production session error: {e}")
    finally:
        bot.stop()

def run_live_production_session(duration_hours: float = 2.0,
                               capital: float = 100,
                               max_daily_loss: float = 200):
    """Run live production session with advanced safety controls"""
    
    print(f"🔴 LIVE PRODUCTION FOREX SESSION")
    print(f"{'='*60}")
    print(f"⚠️  WARNING: This will trade with REAL MONEY")
    print(f"Duration: {duration_hours} hours")
    print(f"Capital: ${capital:,}")
    print(f"Max Daily Loss: ${max_daily_loss}")
    print(f"{'='*60}")
    
    # Safety confirmation (remove in actual production)
    confirm = input("Type 'CONFIRM' to proceed with live trading: ")
    if confirm != 'CONFIRM':
        print("Live trading cancelled")
        return
    
    # Create live bot
    bot = create_advanced_production_forex_bot(
        environment='live',  # LIVE TRADING
        pairs=['EURUSD', 'GBPUSD'],  # Conservative pair selection for live
        capital=capital,
        risk_level='moderate'
    )
    
    if not bot:
        print("Failed to create live trading bot")
        return
    
    try:
        # Additional safety parameters for live trading
        bot.strategy.max_position_pct = 0.10  # More conservative for live
        daily_loss_limit = max_daily_loss
        session_start_equity = capital
        
        print(f"\n🟢 LIVE SESSION STARTED")
        print(f"Safety Limits: Max loss ${daily_loss_limit}, Max exposure 10%")
        
        start_time = datetime.now()
        cycle = 0
        safety_checks = 0
        
        while (datetime.now() - start_time).total_seconds() < duration_hours * 3600:
            cycle += 1
            safety_checks += 1
            
            # Safety check every cycle in live trading
            current_pnl = bot.strategy.total_pnl
            if current_pnl <= -daily_loss_limit:
                print(f"\n🛑 DAILY LOSS LIMIT REACHED: ${current_pnl:.2f}")
                print("Closing all positions and stopping trading...")
                emergency_close_all_advanced_positions(bot)
                break
            
            # Run trading
            bot.run_advanced_trading_cycle()
            
            # More frequent monitoring for live trading
            if cycle % 3 == 0:
                elapsed = (datetime.now() - start_time).total_seconds() / 3600
                perf = bot.strategy.get_advanced_performance_summary()
                
                print(f"\n🔴 LIVE Update (Cycle {cycle}, {elapsed:.1f}h):")
                print(f"  Current P&L: ${perf['total_pnl']:+.2f}")
                print(f"  Available Capital: ${bot.available_capital:,.2f}")
                print(f"  Active Positions: {sum(1 for p in perf['current_positions'].values() if p['quantity'] > 0)}")
                
                # Risk monitoring
                exposure = 0
                for pos in perf['current_positions'].values():
                    if pos['quantity'] > 0:
                        exposure += pos['quantity'] * pos['avg_price']
                
                exposure_pct = (exposure / capital) * 100
                print(f"  Capital Exposure: {exposure_pct:.1f}%")
                
                if exposure_pct > 25:
                    print("  ⚠️ HIGH EXPOSURE WARNING")
            
            # Live trading uses longer cycles for stability
            time.sleep(120)  # 2-minute cycles for live trading
        
        print(f"\n🏁 LIVE PRODUCTION SESSION COMPLETE")
        show_advanced_performance(bot)
        
    except Exception as e:
        print(f"Live session error: {e}")
        print("Attempting emergency position closure...")
        emergency_close_all_advanced_positions(bot)
    finally:
        bot.stop()

def emergency_close_all_advanced_positions(bot: AdvancedForexDayTradingBot):
    """Emergency closure of all positions with detailed logging"""
    
    if not bot or not bot.running:
        print("Bot not available for emergency stop")
        return
    
    print("🚨 EMERGENCY: Closing all advanced positions...")
    
    try:
        positions_closed = 0
        total_pnl = 0
        
        for pair in bot.pairs:
            position = bot.strategy.get_position(pair)
            if position['quantity'] > 0:
                market_data = bot.market_data.get_forex_price(pair)
                if market_data:
                    price = market_data['last']
                    entry_price = position['avg_price']
                    quantity = position['quantity']
                    
                    # Calculate final P&L
                    pip_size = 0.0001 if 'JPY' not in pair else 0.01
                    pnl_pips = (price - entry_price) / pip_size
                    pnl_usd = (price - entry_price) * quantity
                    
                    print(f"  Closing {pair}:")
                    print(f"    Quantity: {quantity:,} units")
                    print(f"    Entry: {entry_price:.5f} → Exit: {price:.5f}")
                    print(f"    P&L: {pnl_pips:+.1f} pips (${pnl_usd:+.2f})")
                    
                    # Execute emergency sell
                    if bot.order_manager.place_forex_market_order(pair, 'SELL', quantity):
                        bot.strategy.update_position(pair, 'SELL', quantity, price)
                        positions_closed += 1
                        total_pnl += pnl_usd
        
        if positions_closed > 0:
            print(f"\n✓ Emergency closure complete:")
            print(f"  Positions closed: {positions_closed}")
            print(f"  Total P&L from closures: ${total_pnl:+.2f}")
        else:
            print("✓ No positions to close")
        
        # Final safety report
        final_perf = bot.strategy.get_advanced_performance_summary()
        print(f"\n📊 Emergency Report:")
        print(f"  Session P&L: ${final_perf['total_pnl']:+.2f}")
        print(f"  Total Trades: {final_perf['total_trades']}")
        print(f"  Emergency Timestamp: {datetime.now().strftime('%H:%M:%S')}")
        
    except Exception as e:
        print(f"Emergency closure error: {e}")

def schedule_advanced_forex_trading(sessions_per_day: int = 3,
                                   session_duration_hours: float = 1.5,
                                   daily_target_return: float = 2.0):
    """Schedule multiple advanced trading sessions throughout the day"""
    
    print(f"=== SCHEDULED ADVANCED FOREX TRADING ===")
    print(f"Sessions per day: {sessions_per_day}")
    print(f"Session duration: {session_duration_hours} hours each")
    print(f"Daily target return: {daily_target_return}%")
    
    daily_pnl = 0
    session_results = []
    
    for session_num in range(1, sessions_per_day + 1):
        print(f"\n🕐 SESSION {session_num}/{sessions_per_day}")
        print(f"Time: {datetime.now().strftime('%H:%M:%S')}")
        
        # Adjust risk based on daily performance
        if daily_pnl < -100:  # If losing day
            risk_level = 'conservative'
            print("📉 Switching to CONSERVATIVE mode (losing day)")
        elif daily_pnl > 150:  # If very profitable day
            risk_level = 'conservative'  # Lock in profits
            print("🔒 Switching to CONSERVATIVE mode (locking profits)")
        else:
            risk_level = 'moderate'
        
        # Run session
        session_start_time = datetime.now()
        
        bot = create_advanced_production_forex_bot(
            environment='paper',
            capital=100 + daily_pnl,  # Adjust capital based on daily P&L
            risk_level=risk_level
        )
        
        if bot:
            try:
                # Run shorter session
                run_duration = min(session_duration_hours, 1.0)  # Max 1 hour per session
                
                start_time = datetime.now()
                cycles = 0
                
                while (datetime.now() - start_time).total_seconds() < run_duration * 3600:
                    cycles += 1
                    bot.run_advanced_trading_cycle()
                    
                    # Check daily target
                    current_session_pnl = bot.strategy.total_pnl
                    projected_daily = daily_pnl + current_session_pnl
                    
                    if projected_daily >= (100 * daily_target_return / 100):
                        print(f"🎯 Daily target reached: ${projected_daily:.2f}")
                        break
                    
                    time.sleep(45)  # 45-second cycles
                
                # Session results
                session_pnl = bot.strategy.total_pnl
                daily_pnl += session_pnl
                
                session_results.append({
                    'session': session_num,
                    'pnl': session_pnl,
                    'trades': bot.strategy.trades_count,
                    'duration': run_duration
                })
                
                print(f"Session {session_num} P&L: ${session_pnl:+.2f}")
                print(f"Daily P&L: ${daily_pnl:+.2f}")
                
            except Exception as e:
                print(f"Session {session_num} error: {e}")
            finally:
                bot.stop()
        
        # Break between sessions (except last one)
        if session_num < sessions_per_day:
            wait_minutes = 30
            print(f"💤 Waiting {wait_minutes} minutes before next session...")
            time.sleep(wait_minutes * 60)
    
    # Daily summary
    print(f"\n📊 DAILY TRADING SUMMARY")
    print(f"{'='*50}")
    print(f"Target Return: {daily_target_return}%")
    print(f"Actual Return: {(daily_pnl/100)*100:+.2f}%")
    print(f"Total P&L: ${daily_pnl:+.2f}")
    
    total_trades = sum(s['trades'] for s in session_results)
    print(f"Total Trades: {total_trades}")
    
    if session_results:
        best_session = max(session_results, key=lambda x: x['pnl'])
        print(f"Best Session: #{best_session['session']} (${best_session['pnl']:+.2f})")
    
    target_met = daily_pnl >= (100 * daily_target_return / 100)
    print(f"Target Met: {'✅ YES' if target_met else '❌ NO'}")

print("✓ Advanced Production Forex Bot created")
print("\nProduction Commands:")
print("• create_advanced_production_forex_bot('paper', risk_level='moderate')")
print("• run_production_advanced_session(2.0) - 2-hour advanced session")
print("• run_live_production_session(1.0) - 1-hour LIVE session")
print("• schedule_advanced_forex_trading(3, 1.5, 2.0) - Daily scheduled trading")
print("• emergency_close_all_advanced_positions(bot) - Emergency stop")

### Week 1: Foundation & Paper Trading Mastery

In [ ]:
# Week 1 - Day 1: Basic Setup and Validation
def week1_day1_setup():
    """Week 1 Day 1: Initial setup and system validation"""
    print("=== WEEK 1 DAY 1: FOUNDATION SETUP ===")
    
    # Test basic connection and data
    quick_forex_price_check(environment='paper', pairs=['EURUSD', 'GBPUSD'])
    
    # Test indicators
    test_forex_indicators('EURUSD', environment='paper')
    
    # Create first bot
    bot = create_advanced_production_forex_bot(
        environment='paper',
        pairs=['EURUSD'],  # Start with single pair
        capital=10000,
        risk_level='conservative'  # Very safe to start
    )
    
    if bot:
        print("✅ Week 1 Day 1 Complete: System validated")
        bot.stop()
    else:
        print("❌ Setup failed - troubleshoot IB connection")

# Week 1 - Day 2-3: Short Test Sessions
def week1_day2_short_sessions():
    """Week 1 Day 2-3: 15-minute test sessions"""
    print("=== WEEK 1 DAY 2-3: SHORT TEST SESSIONS ===")
    
    # Run 3 short sessions
    for session in range(1, 4):
        print(f"\n📚 Learning Session {session}/3")
        
        # Create bot for 15-minute session
        bot = create_advanced_production_forex_bot(
            environment='paper',
            pairs=['EURUSD'],
            capital=10000,
            risk_level='conservative'
        )
        
        if bot:
            # Run 15-minute session manually
            start_time = datetime.now()
            cycles = 0
            
            while (datetime.now() - start_time).total_seconds() < 900:  # 15 minutes
                cycles += 1
                bot.run_advanced_trading_cycle()
                
                if cycles % 5 == 0:
                    perf = bot.strategy.get_advanced_performance_summary()
                    print(f"  Cycle {cycles}: {perf['total_trades']} trades, £{perf['total_pnl']:+.2f}")
                
                time.sleep(30)  # 30-second cycles
            
            bot.stop()
        
        if session < 3:
            print("⏳ 30-minute break between sessions...")
            time.sleep(1800)  # 30-minute break
    
    print("✅ Week 1 Day 2-3 Complete: Basic trading experience gained")

# Week 1 - Day 4-5: Extended Paper Trading
def week1_day4_extended_sessions():
    """Week 1 Day 4-5: 1-hour paper trading sessions"""
    print("=== WEEK 1 DAY 4-5: EXTENDED PAPER SESSIONS ===")
    
    # Morning session
    print("🌅 Morning Session (Conservative)")
    run_production_advanced_session(
        duration_hours=1.0,
        risk_level='conservative'
    )
    
    print("⏳ 2-hour break...")
    time.sleep(7200)  # 2-hour break
    
    # Afternoon session
    print("🌆 Afternoon Session (Moderate)")
    run_production_advanced_session(
        duration_hours=1.0,
        risk_level='moderate'
    )
    
    print("✅ Week 1 Complete: Paper trading foundation established")

# Week 1 Summary Command
def complete_week1():
    """Complete Week 1 training program"""
    print("🎯 WEEK 1: FOUNDATION TRAINING")
    week1_day1_setup()
    time.sleep(3600)  # 1-hour break
    week1_day2_short_sessions()
    time.sleep(7200)  # 2-hour break
    week1_day4_extended_sessions()

print("✅ Week 1 Commands Ready")

### Week 2: Intermediate Strategy Development

In [ ]:
# Week 2 - Day 1-2: Multi-Pair Trading
def week2_day1_multi_pair():
    """Week 2 Day 1-2: Multi-pair trading development"""
    print("=== WEEK 2 DAY 1-2: MULTI-PAIR TRADING ===")
    
    # Session 1: EUR pairs focus
    print("💶 EUR Pairs Session")
    run_production_advanced_session(
        duration_hours=1.5,
        risk_level='moderate'
    )
    
    time.sleep(3600)  # 1-hour break
    
    # Session 2: GBP pairs focus
    print("💷 GBP Pairs Session")
    bot = create_advanced_production_forex_bot(
        environment='paper',
        pairs=['GBPUSD', 'EURGBP'],
        capital=10000,
        risk_level='moderate'
    )
    
    if bot:
        # Custom 90-minute session
        start_time = datetime.now()
        cycle = 0
        
        while (datetime.now() - start_time).total_seconds() < 5400:  # 90 minutes
            cycle += 1
            bot.run_advanced_trading_cycle()
            
            if cycle % 5 == 0:
                show_advanced_performance(bot)
            
            time.sleep(45)
        
        bot.stop()
    
    print("✅ Week 2 Day 1-2 Complete: Multi-pair experience gained")

# Week 2 - Day 3-4: Risk Level Progression
def week2_day3_risk_progression():
    """Week 2 Day 3-4: Progressive risk level testing"""
    print("=== WEEK 2 DAY 3-4: RISK PROGRESSION ===")
    
    risk_levels = ['conservative', 'moderate', 'aggressive']
    
    for i, risk in enumerate(risk_levels):
        print(f"\n📊 Risk Level {i+1}/3: {risk.upper()}")
        
        run_production_advanced_session(
            duration_hours=1.0,
            risk_level=risk
        )
        
        if i < len(risk_levels) - 1:
            print("⏳ 90-minute break between risk levels...")
            time.sleep(5400)  # 90-minute break
    
    print("✅ Week 2 Day 3-4 Complete: Risk management mastered")

# Week 2 - Day 5: Full Day Simulation
def week2_day5_full_simulation():
    """Week 2 Day 5: Full trading day simulation"""
    print("=== WEEK 2 DAY 5: FULL DAY SIMULATION ===")
    
    # Manual implementation since schedule_advanced_forex_trading may not exist
    sessions = 4
    session_duration = 1.0
    daily_target = 1.5
    
    daily_pnl = 0
    
    for session_num in range(1, sessions + 1):
        print(f"\n📅 Session {session_num}/{sessions}")
        
        # Adjust risk based on daily performance
        if daily_pnl < -50:
            risk_level = 'conservative'
        elif daily_pnl > 100:
            risk_level = 'conservative'  # Lock in profits
        else:
            risk_level = 'moderate'
        
        # Run session
        bot = create_advanced_production_forex_bot(
            environment='paper',
            pairs=['EURUSD', 'GBPUSD'],
            capital=10000 + daily_pnl,
            risk_level=risk_level
        )
        
        if bot:
            # Run 1-hour session
            start_time = datetime.now()
            
            while (datetime.now() - start_time).total_seconds() < 3600:  # 1 hour
                bot.run_advanced_trading_cycle()
                time.sleep(60)  # 1-minute cycles
            
            session_pnl = bot.strategy.total_pnl
            daily_pnl += session_pnl
            
            print(f"Session {session_num} P&L: £{session_pnl:+.2f}")
            print(f"Daily P&L: £{daily_pnl:+.2f}")
            
            bot.stop()
        
        # Break between sessions
        if session_num < sessions:
            print("⏳ 1-hour break...")
            time.sleep(3600)
    
    print(f"✅ Day 5 Complete: Daily P&L £{daily_pnl:+.2f}")
    print("✅ Week 2 Complete: Intermediate trading skills developed")

# Week 2 Summary
def complete_week2():
    """Complete Week 2 training program"""
    print("🎯 WEEK 2: INTERMEDIATE DEVELOPMENT")
    week2_day1_multi_pair()
    time.sleep(14400)  # 4-hour break
    week2_day3_risk_progression()
    time.sleep(28800)  # 8-hour overnight break
    week2_day5_full_simulation()

print("✅ Week 2 Commands Ready")

### Week 3: Advanced Strategy Optimisation

In [ ]:
# Week 3 - Day 1-2: High-Frequency Trading
def week3_day1_high_frequency():
    """Week 3 Day 1-2: High-frequency trading optimisation"""
    print("=== WEEK 3 DAY 1-2: HIGH-FREQUENCY OPTIMISATION ===")
    
    # Create high-frequency bot
    bot = create_advanced_production_forex_bot(
        environment='paper',
        pairs=['EURUSD', 'GBPUSD'],
        capital=10000,
        risk_level='aggressive'
    )
    
    if bot:
        # Adjust for high-frequency
        bot.strategy.base_stop_pips = 8      # Tighter stops
        bot.strategy.base_profit_pips = 16   # Smaller profits
        bot.strategy.momentum_threshold = 0.05  # More sensitive
        
        print("⚡ High-Frequency Parameters:")
        print(f"  Stop Loss: {bot.strategy.base_stop_pips} pips")
        print(f"  Take Profit: {bot.strategy.base_profit_pips} pips")
        print(f"  Cycle Frequency: 15 seconds")
        
        # Run intensive 2-hour session
        start_time = datetime.now()
        cycle = 0
        
        while (datetime.now() - start_time).total_seconds() < 7200:  # 2 hours
            cycle += 1
            bot.run_advanced_trading_cycle()
            
            # Show stats every 20 cycles
            if cycle % 20 == 0:
                elapsed = (datetime.now() - start_time).total_seconds() / 3600
                perf = bot.strategy.get_advanced_performance_summary()
                print(f"⚡ HF Update: {cycle} cycles, {elapsed:.1f}h, "
                      f"{perf['total_trades']} trades, £{perf['total_pnl']:+.2f}")
            
            time.sleep(15)  # 15-second cycles
        
        show_advanced_performance(bot)
        bot.stop()
    
    print("✅ Week 3 Day 1-2 Complete: High-frequency mastery achieved")

# Week 3 - Day 3-4: Multi-Session Strategy
def week3_day3_multi_session():
    """Week 3 Day 3-4: Advanced multi-session trading"""
    print("=== WEEK 3 DAY 3-4: MULTI-SESSION MASTERY ===")
    
    # Manual implementation of multi-session trading
    sessions_per_day = 6
    session_duration_hours = 0.75  # 45-minute sessions
    daily_target_return = 3.0
    
    daily_pnl = 0
    session_results = []
    
    for session_num in range(1, sessions_per_day + 1):
        print(f"\n🕐 SESSION {session_num}/{sessions_per_day}")
        print(f"Time: {datetime.now().strftime('%H:%M:%S')}")
        
        # Adjust risk based on daily performance
        if daily_pnl < -100:
            risk_level = 'conservative'
            print("📉 Switching to CONSERVATIVE mode (losing day)")
        elif daily_pnl > 150:
            risk_level = 'conservative'  # Lock in profits
            print("🔒 Switching to CONSERVATIVE mode (locking profits)")
        else:
            risk_level = 'moderate'
        
        # Create bot for this session
        bot = create_advanced_production_forex_bot(
            environment='paper',
            pairs=['EURUSD', 'GBPUSD', 'USDJPY'],
            capital=10000 + daily_pnl,
            risk_level=risk_level
        )
        
        if bot:
            # Run 45-minute session
            start_time = datetime.now()
            cycles = 0
            
            while (datetime.now() - start_time).total_seconds() < 2700:  # 45 minutes
                cycles += 1
                bot.run_advanced_trading_cycle()
                
                # Check daily target
                current_session_pnl = bot.strategy.total_pnl
                projected_daily = daily_pnl + current_session_pnl
                
                if projected_daily >= (100 * daily_target_return / 100):
                    print(f"🎯 Daily target reached: £{projected_daily:.2f}")
                    break
                
                time.sleep(30)  # 30-second cycles
            
            # Session results
            session_pnl = bot.strategy.total_pnl
            daily_pnl += session_pnl
            
            session_results.append({
                'session': session_num,
                'pnl': session_pnl,
                'trades': bot.strategy.trades_count,
                'duration': session_duration_hours
            })
            
            print(f"Session {session_num} P&L: £{session_pnl:+.2f}")
            print(f"Daily P&L: £{daily_pnl:+.2f}")
            
            bot.stop()
        
        # Break between sessions (except last one)
        if session_num < sessions_per_day:
            wait_minutes = 15
            print(f"💤 Waiting {wait_minutes} minutes before next session...")
            time.sleep(wait_minutes * 60)
    
    # Daily summary
    print(f"\n📊 DAILY SUMMARY")
    print(f"Target Return: {daily_target_return}%")
    print(f"Actual Return: {(daily_pnl/100)*100:+.2f}%")
    print(f"Total P&L: £{daily_pnl:+.2f}")
    
    total_trades = sum(s['trades'] for s in session_results)
    print(f"Total Trades: {total_trades}")
    
    if session_results:
        best_session = max(session_results, key=lambda x: x['pnl'])
        print(f"Best Session: #{best_session['session']} (£{best_session['pnl']:+.2f})")
    
    target_met = daily_pnl >= (100 * daily_target_return / 100)
    print(f"Target Met: {'✅ YES' if target_met else '❌ NO'}")
    
    print("✅ Week 3 Day 3-4 Complete: Multi-session expertise developed")

# Week 3 - Day 5: Performance Optimisation
def week3_day5_optimisation():
    """Week 3 Day 5: Advanced performance optimisation"""
    print("=== WEEK 3 DAY 5: PERFORMANCE OPTIMISATION ===")
    
    # Test different capital amounts
    capital_levels = [5000, 100, 15000]
    
    for capital in capital_levels:
        print(f"\n💰 Testing with £{capital:,} capital")
        
        run_production_advanced_session(
            duration_hours=1.0,
            risk_level='moderate'
        )
        
        time.sleep(1800)  # 30-minute break between tests
    
    print("✅ Week 3 Complete: Advanced optimisation mastered")

# Week 3 Summary
def complete_week3():
    """Complete Week 3 advanced training"""
    print("🎯 WEEK 3: ADVANCED OPTIMISATION")
    week3_day1_high_frequency()
    time.sleep(21600)  # 6-hour break
    week3_day3_multi_session()
    time.sleep(28800)  # 8-hour break
    week3_day5_optimisation()

print("✅ Week 3 Commands Ready")

### Week 4: Live Trading Preparation & Execution

In [ ]:
# Week 4 - Day 1-2: Pre-Live Validation
def week4_day1_pre_live_validation():
    """Week 4 Day 1-2: Final validation before live trading"""
    print("=== WEEK 4 DAY 1-2: PRE-LIVE VALIDATION ===")
    
    # Intensive paper trading with live-like conditions
    print("🔬 Live Simulation Test 1: Conservative")
    run_production_advanced_session(
        duration_hours=2.0,
        risk_level='conservative'
    )
    
    time.sleep(7200)  # 2-hour break
    
    print("🔬 Live Simulation Test 2: Moderate")
    run_production_advanced_session(
        duration_hours=2.0,
        risk_level='moderate'
    )
    
    print("✅ Pre-live validation complete")

# Week 4 - Day 3: First Live Trading (Ultra Conservative)
def week4_day3_first_live():
    """Week 4 Day 3: First live trading session"""
    print("=== WEEK 4 DAY 3: FIRST LIVE TRADING ===")
    print("🔴 CAUTION: Real money trading begins")
    
    # Manual implementation for ultra-conservative live session
    print("⚠️ WARNING: This is a simulation of live trading")
    print("For actual live trading, change environment to 'live'")
    
    # Create ultra-conservative bot (using paper for safety)
    bot = create_advanced_production_forex_bot(
        environment='live',  # Keep as paper for safety
        pairs=['EURUSD'],     # Single pair only
        capital=10000,         # Small capital
        risk_level='conservative'
    )
    
    if bot:
        # Ultra-conservative parameters
        bot.strategy.base_position_pct = 0.03  # 3% positions only
        bot.strategy.max_position_pct = 0.10   # 10% max exposure
        bot.strategy.base_stop_pips = 25       # Wide stops
        
        print("🛡️ Ultra-Conservative Parameters:")
        print(f"  Position Size: {bot.strategy.base_position_pct*100:.0f}% of capital")
        print(f"  Max Exposure: {bot.strategy.max_position_pct*100:.0f}%")
        print(f"  Stop Loss: {bot.strategy.base_stop_pips} pips")
        
        # Run 30-minute session
        start_time = datetime.now()
        max_loss = 50
        
        while (datetime.now() - start_time).total_seconds() < 1800:  # 30 minutes
            bot.run_advanced_trading_cycle()
            
            # Safety check every cycle
            current_pnl = bot.strategy.total_pnl
            if current_pnl <= -max_loss:
                print(f"🛑 LOSS LIMIT REACHED: £{current_pnl:.2f}")
                break
            
            time.sleep(60)  # 1-minute cycles for safety
        
        show_advanced_performance(bot)
        bot.stop()
    
    print("✅ First live session complete")

# Week 4 - Day 4: Scaled Live Trading
def week4_day4_scaled_live():
    """Week 4 Day 4: Scaled live trading"""
    print("=== WEEK 4 DAY 4: SCALED LIVE TRADING ===")
    
    # Scaled live session (simulated)
    print("📈 Increased capital and duration")
    
    bot = create_advanced_production_forex_bot(
        environment='live',  # Keep as paper for safety
        pairs=['EURUSD', 'GBPUSD'],
        capital=25000,
        risk_level='moderate'
    )
    
    if bot:
        # Moderate parameters
        bot.strategy.base_position_pct = 0.06  # 6% positions
        bot.strategy.max_position_pct = 0.20   # 20% max exposure
        
        print("📊 Scaled Parameters:")
        print(f"  Capital: £{bot.available_capital:,}")
        print(f"  Position Size: {bot.strategy.base_position_pct*100:.0f}%")
        print(f"  Max Exposure: {bot.strategy.max_position_pct*100:.0f}%")
        
        # Run 1-hour session
        start_time = datetime.now()
        max_loss = 100
        
        while (datetime.now() - start_time).total_seconds() < 3600:  # 1 hour
            bot.run_advanced_trading_cycle()
            
            # Safety monitoring
            current_pnl = bot.strategy.total_pnl
            if current_pnl <= -max_loss:
                print(f"🛑 LOSS LIMIT REACHED: £{current_pnl:.2f}")
                break
            
            time.sleep(45)  # 45-second cycles
        
        show_advanced_performance(bot)
        bot.stop()
    
    print("✅ Scaled live trading complete")

# Week 4 - Day 5: Full Production Live Trading
def week4_day5_full_production():
    """Week 4 Day 5: Full production live trading"""
    print("=== WEEK 4 DAY 5: FULL PRODUCTION TRADING ===")
    
    # Full production session (simulated)
    print("🚀 Full production parameters")
    print("⚠️ For actual live trading, change environment to 'live'")
    
    bot = create_advanced_production_forex_bot(
        environment='live',  # Change to 'live' for real trading
        pairs=['EURUSD', 'GBPUSD', 'USDJPY'],
        capital=50000,
        risk_level='moderate'
    )
    
    if bot:
        print("💼 Production Parameters:")
        print(f"  Capital: £{bot.available_capital:,}")
        print(f"  Trading Pairs: {', '.join(bot.pairs)}")
        print(f"  Risk Level: MODERATE")
        
        # Run 2-hour production session
        start_time = datetime.now()
        max_loss = 200
        cycle = 0
        
        while (datetime.now() - start_time).total_seconds() < 7200:  # 2 hours
            cycle += 1
            bot.run_advanced_trading_cycle()
            
            # Production monitoring every 10 cycles
            if cycle % 10 == 0:
                elapsed = (datetime.now() - start_time).total_seconds() / 3600
                perf = bot.strategy.get_advanced_performance_summary()
                
                print(f"🔴 Production Update (Cycle {cycle}, {elapsed:.1f}h):")
                print(f"  P&L: £{perf['total_pnl']:+.2f}")
                print(f"  Trades: {perf['total_trades']} | Win Rate: {perf['win_rate']:.0f}%")
                
                # Risk monitoring
                if perf['total_pnl'] <= -max_loss:
                    print(f"🛑 DAILY LOSS LIMIT: £{perf['total_pnl']:.2f}")
                    break
            
            time.sleep(30)  # 30-second cycles
        
        show_advanced_performance(bot)
        bot.stop()
    
    print("✅ Full production trading complete")

# Week 4 Summary
def complete_week4():
    """Complete Week 4 live trading progression"""
    print("🎯 WEEK 4: LIVE TRADING MASTERY")
    week4_day1_pre_live_validation()
    time.sleep(86400)  # 24-hour break before live
    week4_day3_first_live()
    time.sleep(86400)  # 24-hour break
    week4_day4_scaled_live()
    time.sleep(86400)  # 24-hour break
    week4_day5_full_production()

print("✅ Week 4 Commands Ready")

### Complete 4-Week Program Execution

In [ ]:
def run_complete_4week_program():
    """Execute the complete 4-week forex trading program"""
    
    print("4-WEEK PROGRESSIVE FOREX TRADING PROGRAM")
    print("="*60)
    print("Week 1: Foundation & Paper Trading (£10,000 virtual)")
    print("Week 2: Intermediate Multi-Pair Strategy")
    print("Week 3: Advanced High-Frequency Optimisation")
    print("Week 4: Live Trading Progression (£1,000→£5,000 real)")
    print("="*60)
    
    # Confirmation for full program
    confirm = input("\nStart complete 4-week program? (Type 'START'): ")
    if confirm != 'START':
        print("Program cancelled")
        return
    
    try:
        # Week 1: Foundation
        print(f"\nStarting Week 1...")
        complete_week1()
        print(f"Week 1 Complete")
        time.sleep(3600)  # 1-hour break (not 7 days!)
        
        # Week 2: Intermediate
        print(f"\nStarting Week 2...")
        complete_week2()
        print(f"Week 2 Complete")
        time.sleep(3600)  # 1-hour break
        
        # Week 3: Advanced
        print(f"\nStarting Week 3...")
        complete_week3()
        print(f"Week 3 Complete")
        time.sleep(3600)  # 1-hour break
        
        # Week 4: Live Trading
        print(f"\nStarting Week 4 (LIVE TRADING)...")
        complete_week4()
        print(f"Week 4 Complete")
        
        print(f"\n4-WEEK PROGRAM COMPLETE!")
        print("You are now ready for professional forex day trading")
        
    except KeyboardInterrupt:
        print("\nProgram interrupted by user")
    except Exception as e:
        print(f"Program error: {e}")

# Individual week functions for flexible execution
def execute_week(week_number: int):
    """Execute a specific week of the program"""
    
    week_functions = {
        1: complete_week1,
        2: complete_week2,
        3: complete_week3,
        4: complete_week4
    }
    
    if week_number in week_functions:
        print(f"Executing Week {week_number}")
        week_functions[week_number]()
    else:
        print(f"Invalid week number: {week_number}")

# Daily practice functions
def daily_practice_session(week: int, day: int):
    """Run daily practice appropriate for current week/day"""
    
    practice_schedule = {
        (1, 1): ("Foundation Setup", week1_day1_setup),
        (1, 2): ("Short Sessions", week1_day2_short_sessions),
        (1, 3): ("Short Sessions", week1_day2_short_sessions),
        (1, 4): ("Extended Sessions", week1_day4_extended_sessions),
        (1, 5): ("Extended Sessions", week1_day4_extended_sessions),
        
        (2, 1): ("Multi-Pair", week2_day1_multi_pair),
        (2, 2): ("Multi-Pair", week2_day1_multi_pair),
        (2, 3): ("Risk Progression", week2_day3_risk_progression),
        (2, 4): ("Risk Progression", week2_day3_risk_progression),
        (2, 5): ("Full Simulation", week2_day5_full_simulation),
        
        (3, 1): ("High Frequency", week3_day1_high_frequency),
        (3, 2): ("High Frequency", week3_day1_high_frequency),
        (3, 3): ("Multi-Session", week3_day3_multi_session),
        (3, 4): ("Multi-Session", week3_day3_multi_session),
        (3, 5): ("Optimisation", week3_day5_optimisation),
        
        (4, 1): ("Pre-Live Validation", week4_day1_pre_live_validation),
        (4, 2): ("Pre-Live Validation", week4_day1_pre_live_validation),
        (4, 3): ("First Live", week4_day3_first_live),
        (4, 4): ("Scaled Live", week4_day4_scaled_live),
        (4, 5): ("Full Production", week4_day5_full_production)
    }
    
    if (week, day) in practice_schedule:
        description, function = practice_schedule[(week, day)]
        print(f"Week {week} Day {day}: {description}")
        function()
    else:
        print(f"No practice scheduled for Week {week} Day {day}")

print("Complete 4-Week Program Ready")

### 4-Week Progressive Trading Commands


In [ ]:
print("="*70)
print("4-WEEK PROGRESSIVE FOREX TRADING PROGRAM")
print("="*70)

print("\nWEEK 1: FOUNDATION (Paper Trading)")
print("Capital: £10,000 virtual | Risk: Conservative")
print("• week1_day1_setup() - Initial setup and validation")
print("• week1_day2_short_sessions() - 15-minute test sessions")
print("• week1_day4_extended_sessions() - 1-hour paper sessions")
print("• complete_week1() - Full Week 1 program")

print("\nWEEK 2: INTERMEDIATE (Multi-Pair Strategy)")
print("Capital: £10,000 virtual | Risk: Conservative → Moderate")
print("• week2_day1_multi_pair() - Multi-pair trading")
print("• week2_day3_risk_progression() - Risk level testing")
print("• week2_day5_full_simulation() - Full day simulation")
print("• complete_week2() - Full Week 2 program")

print("\nWEEK 3: ADVANCED (High-Frequency Optimisation)")
print("Capital: £10,000+ virtual | Risk: Moderate → Aggressive")
print("• week3_day1_high_frequency() - High-frequency trading")
print("• week3_day3_multi_session() - 6 sessions per day")
print("• week3_day5_optimisation() - Performance optimisation")
print("• complete_week3() - Full Week 3 program")

print("\nWEEK 4: LIVE TRADING (Real Money Progression)")
print("Capital: £1,000 → £5,000 REAL | Risk: Ultra Conservative → Moderate")
print("• week4_day1_pre_live_validation() - Final paper validation")
print("• week4_day3_first_live() - First live session (£1,000, 30min)")
print("• week4_day4_scaled_live() - Scaled live (£2,500, 1hour)")
print("• week4_day5_full_production() - Full production (£5,000, 2hours)")
print("• complete_week4() - Full Week 4 program")

print("\nQUICK EXECUTION:")
print("• execute_week(1) - Run specific week")
print("• daily_practice_session(2, 3) - Run specific day")
print("• run_complete_4week_program() - Full 4-week program")

print("\nRECOMMENDED PROGRESSION:")
print("Day 1-5: execute_week(1)")
print("Day 8-12: execute_week(2)")
print("Day 15-19: execute_week(3)")
print("Day 22-26: execute_week(4)")

print("\nIMPORTANT NOTES:")
print("• Week 1-3: Paper trading only (no real money)")
print("• Week 4: Gradual real money introduction")
print("• Each week builds on previous skills")
print("• Take breaks between intensive sessions")
print("• Monitor performance and adjust as needed")

print("="*70)
print("START YOUR JOURNEY: execute_week(1)")
print("="*70)

### Trade Execution

In [ ]:
# Uncomment to run specific functions

# === WEEK 1: FOUNDATION ===
# week1_day1_setup()
week1_day2_short_sessions()
# week1_day4_extended_sessions()
# complete_week1()

# === WEEK 2: INTERMEDIATE ===
# week2_day1_multi_pair()
# week2_day3_risk_progression()
# week2_day5_full_simulation()
# complete_week2()

# === WEEK 3: ADVANCED ===
# week3_day1_high_frequency()
# week3_day3_multi_session()
# week3_day5_optimisation()
# complete_week3()

# === WEEK 4: LIVE TRADING ===
# week4_day1_pre_live_validation()
# week4_day3_first_live()
# week4_day4_scaled_live()
# week4_day5_full_production()
# complete_week4()

# === FULL PROGRAM EXECUTION ===
# execute_week(1)
# execute_week(2)
# execute_week(3)
# execute_week(4)
# run_complete_4week_program()

# === INDIVIDUAL DAY PRACTICE ===
# daily_practice_session(1, 1)  # Week 1, Day 1
# daily_practice_session(2, 3)  # Week 2, Day 3
# daily_practice_session(3, 5)  # Week 3, Day 5
# daily_practice_session(4, 3)  # Week 4, Day 3

# === QUICK TESTS OR UTILITY FUNCTIONS ===
# quick_forex_price_check()
# test_forex_indicators('EURUSD')
# demo_advanced_forex_trading()

### Capital Progression Summary

In [ ]:
def show_capital_progression():
    """Show the capital progression through 4 weeks"""
    
    print("CAPITAL PROGRESSION THROUGH 4 WEEKS")
    print("="*50)
    
    weeks = [
        {
            'week': 1,
            'type': 'Paper Trading',
            'capital': '£10,000 virtual',
            'risk': 'Conservative (5% positions)',
            'max_loss': '£500 virtual',
            'target_daily': '0.5%',
            'sessions': '2-3 per day, 15-60 minutes'
        },
        {
            'week': 2,
            'type': 'Paper Trading',
            'capital': '£10,000 virtual',
            'risk': 'Moderate (8% positions)',
            'max_loss': '£800 virtual',
            'target_daily': '1.0%',
            'sessions': '3-4 per day, 60-90 minutes'
        },
        {
            'week': 3,
            'type': 'Paper Trading',
            'capital': '£10,000+ virtual',
            'risk': 'Aggressive (12% positions)',
            'max_loss': '£1,200 virtual',
            'target_daily': '2.0%',
            'sessions': '4-6 per day, 45-120 minutes'
        },
        {
            'week': 4,
            'type': 'LIVE TRADING',
            'capital': '£1,000 → £5,000 REAL',
            'risk': 'Conservative → Moderate',
            'max_loss': '£50 → £200 REAL',
            'target_daily': '1.0% → 2.0%',
            'sessions': '1-2 per day, 30-120 minutes'
        }
    ]
    
    for week_info in weeks:
        print(f"\nWEEK {week_info['week']}: {week_info['type']}")
        print(f"  Capital: {week_info['capital']}")
        print(f"  Risk Level: {week_info['risk']}")
        print(f"  Max Daily Loss: {week_info['max_loss']}")
        print(f"  Target Return: {week_info['target_daily']}")
        print(f"  Sessions: {week_info['sessions']}")
    
    print(f"\nTOTAL PROGRAMME INVESTMENT:")
    print(f"  Time: 4 weeks (20 trading days)")
    print(f"  Paper Capital: £10,000 virtual")
    print(f"  Live Capital: £1,000 → £5,000 real")
    print(f"  Maximum Risk: £200 real money")
    
    print(f"\nEXPECTED OUTCOMES:")
    print(f"  • Master forex day trading fundamentals")
    print(f"  • Develop risk management discipline")
    print(f"  • Build confidence with real money")
    print(f"  • Establish profitable trading routine")
    print(f"  • Ready for independent trading")

# 4-WEEK PROGRESSIVE TRADING COMMANDS SUMMARY
print("="*70)
print("4-WEEK PROGRESSIVE FOREX TRADING PROGRAMME")
print("="*70)

# === UTILITY FUNCTIONS ===
show_capital_progression()

### Day Trading Programme

In [ ]:
print("\nWEEK 1: FOUNDATION (Paper Trading)")
print("Capital: £10,000 virtual | Risk: Conservative")
print("• week1_day1_setup() - Initial setup and validation")
print("• week1_day2_short_sessions() - 15-minute test sessions")
print("• week1_day4_extended_sessions() - 1-hour paper sessions")
print("• complete_week1() - Full Week 1 programme")

print("\nWEEK 2: INTERMEDIATE (Multi-Pair Strategy)")
print("Capital: £10,000 virtual | Risk: Conservative → Moderate")
print("• week2_day1_multi_pair() - Multi-pair trading")
print("• week2_day3_risk_progression() - Risk level testing")
print("• week2_day5_full_simulation() - Full day simulation")
print("• complete_week2() - Full Week 2 programme")

print("\nWEEK 3: ADVANCED (High-Frequency Optimisation)")
print("Capital: £10,000+ virtual | Risk: Moderate → Aggressive")
print("• week3_day1_high_frequency() - High-frequency trading")
print("• week3_day3_multi_session() - 6 sessions per day")
print("• week3_day5_optimisation() - Performance optimisation")
print("• complete_week3() - Full Week 3 programme")

print("\nWEEK 4: LIVE TRADING (Real Money Progression)")
print("Capital: £1,000 → £5,000 REAL | Risk: Ultra Conservative → Moderate")
print("• week4_day1_pre_live_validation() - Final paper validation")
print("• week4_day3_first_live() - First live session (£1,000, 30min)")
print("• week4_day4_scaled_live() - Scaled live (£2,500, 1hour)")
print("• week4_day5_full_production() - Full production (£5,000, 2hours)")
print("• complete_week4() - Full Week 4 programme")

print("Ready to begin your forex trading journey!")